In [2]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


#edge
from webdriver_manager.microsoft import EdgeChromiumDriverManager #edge時(預設瀏覽器)
from selenium.webdriver.edge.service import Service

import time
import csv


driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
# 最大化瀏覽器視窗
driver.maximize_window()
#________________________________________________________________________________

data_list1 = []
data1={}#使用大刮號{}創建字典
url='https://news.cnyes.com/news/cat/tw_stock_news?exp=a'
driver.get(url)

date=driver.find_element(By.CLASS_NAME,'_Qfx4')
date.click()
div_1=2 #1or2
div_2=2 #2~7
div_3=7 #1~7
i=0;
# 初始化列表用於存儲日期和標題

#日期往前按20頁
for i in range(20):
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[1]/div[1]/div[1]'))).click()
#1月
for i in range(31):
    try:
        print(f"{div_1}{div_2}{div_3}")
            
        if(i!=0):
            # 使用 JavaScript 進行滾動
            driver.execute_script("window.scrollBy(0, 300);")
            # 選擇日期選擇器
            date_selector = driver.find_element(By.CLASS_NAME, '_Qfx4')
            date_selector.click()
            
        # 暫時的變數
        temp_div_1 = div_1
        temp_div_2 = div_2
        temp_div_3 = div_3

        # 選擇1/1    
        start = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(start))
        start2 = driver.find_element(*start)
        action = ActionChains(driver)
        action.move_to_element(start2).click().perform()
#日期往後按10頁
#for i in range(10):
    #nextpage=driver.find_element(By.XPATH,'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]').click()

        # 選擇1/1
        end = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(end))
        end2 = driver.find_element(*end)
        action2 = ActionChains(driver)
        action2.move_to_element(end2).click().perform()
        #確定按鈕
        button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[2]/button[1]')
        driver.execute_script("arguments[0].click();", button)

        time.sleep(5)

    
#模擬向下捲動，以確保載入更多內容
        scrolls = 3
        for _ in range(scrolls):
            driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
            time.sleep(2)  # 等待頁面載入
#找到當前頁面
        page_content=driver.page_source

# 使用 BeautifulSoup 解析頁面内容
        soup = BeautifulSoup(page_content, "html.parser")

        # 抓取日期元素
        date_elements = soup.select("div._67tN.theme-meta time")

        # 抓取標題元素
        title_elements = soup.select("div._1xc2 h3")

        # 將日期和標題對應成字典，然後添加到列表中
        for date_element, title_element in zip(date_elements, title_elements):
            data1 = {
                'date': date_element.text,
                'title': title_element.text
            }
            data_list1.append(data1)

        i=i+1
        if temp_div_3 == 7:
            temp_div_3 = 1
            if temp_div_2 == 7:
                temp_div_2 = 2
                if temp_div_1 == 2:
                    nextpage = driver.find_element(By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]')
                    nextpage.click()
                    temp_div_1 = 1
                else:
                    temp_div_1 = temp_div_1 + 1
            else:
                temp_div_2 = temp_div_2 + 1
        else:
            temp_div_3 = temp_div_3 + 1

        # 此輪迴圈結束後，將暫時的變數賦值給 div_1、div_2 和 div_3
        div_1 = temp_div_1
        div_2 = temp_div_2
        div_3 = temp_div_3
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
    except NoSuchElementException:
        if(div_3==7):
            div_3=1
            if(div_2==7):
                div_2=2
                if(div_1==2):
                    nextpage=driver.find_element(By.XPATH,'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]').click()
                    div_1=1
                else:
                    div_1=div_1+1
            else:
                div_2=div_2+1
        else:
            div_3=div_3+1
        date3=driver.find_element(By.CLASS_NAME,'_Qfx4')
        date3.click()
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
driver.close()
# 輸出所有日期和標題對應的資料
for data1 in data_list1:
    print(data1['date'], data1['title'])
            
with open('新聞標題(2022~2023)/news_1.csv', mode='w', encoding='utf-8', newline='') as csvfile:
    # 定義欄位
    fieldnames = ['date', 'title']

    # 將 dictionary 寫入 CSV 檔
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # 寫入第一列的欄位名稱
    writer.writeheader()
    for data1 in data_list1:
        # 寫入資料
        writer.writerow({'date': data1['date'], 'title': data1['title']})


227


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: MicrosoftEdge=116.0.1938.69)
Stacktrace:
	GetHandleVerifier [0x00007FF6EA833D72+61490]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF6EA7C5FC2+740594]
	(No symbol) [0x00007FF6EA59B8AE]
	(No symbol) [0x00007FF6EA572C1E]
	(No symbol) [0x00007FF6EA60785D]
	(No symbol) [0x00007FF6EA61C758]
	(No symbol) [0x00007FF6EA6036E3]
	(No symbol) [0x00007FF6EA5D5EAA]
	(No symbol) [0x00007FF6EA5D518B]
	(No symbol) [0x00007FF6EA5D6634]
	Microsoft::Applications::Events::ILogManager::DispatchEventBroadcast [0x00007FF6EA9F8D29+1207369]
	(No symbol) [0x00007FF6EA655304]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF6EA7190E1+32273]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF6EA7119D9+1801]
	Microsoft::Applications::Events::ILogManager::DispatchEventBroadcast [0x00007FF6EA9F7904+1202212]
	Mi

In [22]:
#2月
driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
data_list2 = []
data2={}#使用大刮號{}創建字典
url='https://news.cnyes.com/news/cat/tw_stock_news?exp=a'
driver.get(url)

date=driver.find_element(By.CLASS_NAME,'_Qfx4')
date.click()
div_1=2 #1or2
div_2=2 #2~7
div_3=3 #1~7
i=0;
#日期往前按19頁
for i in range(19):
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[1]/div[1]/div[1]'))).click()
#2月
for i in range(28):
    try:
        # 初始化列表用於存儲日期和標題
        data_list = []
        print(f"{div_1}{div_2}{div_3}")
        if(i!=0):
            # 使用 JavaScript 進行滾動
            driver.execute_script("window.scrollBy(0, 300);")
            # 選擇日期選擇器
            date_selector = driver.find_element(By.CLASS_NAME, '_Qfx4')
            date_selector.click()
        # 暫時的變數
        temp_div_1 = div_1
        temp_div_2 = div_2
        temp_div_3 = div_3

        # 選擇2/1    
        start = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(start))
        start2 = driver.find_element(*start)
        action = ActionChains(driver)
        action.move_to_element(start2).click().perform()
        # 選擇2/1
        end = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(end))
        end2 = driver.find_element(*end)
        action2 = ActionChains(driver)
        action2.move_to_element(end2).click().perform()
        #確定按鈕
        button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[2]/button[1]')
        driver.execute_script("arguments[0].click();", button)

        time.sleep(5)

    
#模擬向下捲動，以確保載入更多內容
        scrolls = 3
        for _ in range(scrolls):
            driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
            time.sleep(2)  # 等待頁面載入
#找到當前頁面
        page_content=driver.page_source

# 使用 BeautifulSoup 解析頁面内容
        soup = BeautifulSoup(page_content, "html.parser")

        # 抓取日期元素
        date_elements = soup.select("div._67tN.theme-meta time")

        # 抓取標題元素
        title_elements = soup.select("div._1xc2 h3")

        # 將日期和標題對應成字典，然後添加到列表中
        for date_element, title_element in zip(date_elements, title_elements):
            data2 = {
                'date': date_element.text,
                'title': title_element.text
            }
            data_list2.append(data2)
        

        i=i+1
        if temp_div_3 == 7:
            temp_div_3 = 1
            if temp_div_2 == 7:
                temp_div_2 = 2
                if temp_div_1 == 2:
                    nextpage = driver.find_element(By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]')
                    nextpage.click()
                    temp_div_1 = 1
                else:
                    temp_div_1 = temp_div_1 + 1
            else:
                temp_div_2 = temp_div_2 + 1
        else:
            temp_div_3 = temp_div_3 + 1

        # 此輪迴圈結束後，將暫時的變數賦值給 div_1、div_2 和 div_3
        div_1 = temp_div_1
        div_2 = temp_div_2
        div_3 = temp_div_3
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
    except NoSuchElementException:
        if(div_3==7):
            div_3=1
            if(div_2==7):
                div_2=2
                if(div_1==2):
                    nextpage=driver.find_element(By.XPATH,'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]').click()
                    div_1=1
                else:
                    div_1=div_1+1
            else:
                div_2=div_2+1
        else:
            div_3=div_3+1
        date3=driver.find_element(By.CLASS_NAME,'_Qfx4')
        date3.click()
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
driver.close()
# 輸出所有日期和標題對應的資料
for data2 in data_list2:
    print(data2['date'], data2['title'])
            
with open('新聞標題(2022~2023)/news_2.csv', mode='w', encoding='utf-8', newline='') as csvfile:
    # 定義欄位
    fieldnames = ['date', 'title']

    # 將 dictionary 寫入 CSV 檔
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # 寫入第一列的欄位名稱
    writer.writeheader()
    for data2 in data_list2:
        # 寫入資料
        writer.writerow({'date': data2['date'], 'title': data2['title']})

223
224
225
226
227
231
232
233
234
235
236
237
241
242
243
244
245
246
247
251
252
253
254
255
256
257
261
262
2022/02/01 〈虎年企業加薪留才〉聯發科、聯詠因應新領域獵數千位人才 調薪、員工福利不手軟
2022/02/01 〈虎年企業加薪留才〉服務業大缺工 四大餐飲集團加薪+釋出近4000個職缺
2022/02/01 〈虎年企業加薪留才〉搶不贏科技大廠 工具機業今年加薪10%優於往年
2022/02/01 〈虎年企業加薪留才〉成衣雙雄分享營運成果 年年加薪不手軟
2022/02/01 〈虎年企業加薪留才〉公股銀強化戰力 年後釋出逾3000名職缺爭取高手助陣
2022/02/01 〈虎年企業加薪留才〉品牌雙A搶各路好手 宏碁今年推員工持股信託福利
2022/02/01 〈虎年企業加薪留才〉元宇宙商機熱 宏達電擴大VR布局 招聘逾百名人才
2022/02/01 環球晶收購世創破局 須支付15.6億元交易終止費
2022/02/01 〈虎年企業加薪留才〉上海商銀調薪4% 台新金最高可望加薪15%
2022/02/02 〈觀察〉三因素支撐 鋼構廠議價能力續強
2022/02/02 〈觀察〉台積電一手扶植本土化學品供應鏈 傳英特爾有意來取經
2022/02/03 〈潛力股〉聯發科旗艦晶片再勝高通 站穩全球手機晶片龍頭
2022/02/03 〈潛力股〉IKKA-KY搭電動車大商機 獲利有望逐年上揚
2022/02/03 〈潛力股〉正新今年EPS挑戰6年新高
2022/02/03 〈潛力股〉志超四川遂寧廠新產能今年加入投產 挹注營運再成長
2022/02/03 〈潛力股〉錸寶PMOLED本業穩 電池儲能業務助攻營運成長
2022/02/03 〈潛力股〉永昕二廠明年開始貢獻業績 營收有望倍增
2022/02/03 〈潛力股〉廣宇電動車布局加持 今年營運成長更顯著
2022/02/03 〈虎年外幣投資術〉美元續強卻非向上突破 可搭配三種外幣布局
2022/02/04 〈Q4財報搶先看〉LED廠各擁動能 光寶科、鼎元全年EPS可望創高
2022/02/04 〈Q4財報搶先看〉保護元件三劍客 去年獲利連袂登峰
2022/02/04 〈Q4財報搶先看〉太陽能模組趕出貨 

In [23]:
#3月
driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
data_list3 = []
data3={}#使用大刮號{}創建字典
url='https://news.cnyes.com/news/cat/tw_stock_news?exp=a'
driver.get(url)

date=driver.find_element(By.CLASS_NAME,'_Qfx4')
date.click()
div_1=2 #1or2
div_2=2 #2~7
div_3=3 #1~7
i=0;
#日期往前按18頁
for i in range(18):
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[1]/div[1]/div[1]'))).click()
#3月
for i in range(31):
    try:
        # 初始化列表用於存儲日期和標題
        data_list = []
        print(f"{div_1}{div_2}{div_3}")
        if(i!=0):
            # 使用 JavaScript 進行滾動
            driver.execute_script("window.scrollBy(0, 300);")
            # 選擇日期選擇器
            date_selector = driver.find_element(By.CLASS_NAME, '_Qfx4')
            date_selector.click()
        # 暫時的變數
        temp_div_1 = div_1
        temp_div_2 = div_2
        temp_div_3 = div_3

        # 選擇3/1    
        start = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(start))
        start2 = driver.find_element(*start)
        action = ActionChains(driver)
        action.move_to_element(start2).click().perform()
        # 選擇3/1
        end = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(end))
        end2 = driver.find_element(*end)
        action2 = ActionChains(driver)
        action2.move_to_element(end2).click().perform()
        #確定按鈕
        button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[2]/button[1]')
        driver.execute_script("arguments[0].click();", button)

        time.sleep(5)

    
#模擬向下捲動，以確保載入更多內容
        scrolls = 3
        for _ in range(scrolls):
            driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
            time.sleep(2)  # 等待頁面載入
#找到當前頁面
        page_content=driver.page_source

# 使用 BeautifulSoup 解析頁面内容
        soup = BeautifulSoup(page_content, "html.parser")

        # 抓取日期元素
        date_elements = soup.select("div._67tN.theme-meta time")

        # 抓取標題元素
        title_elements = soup.select("div._1xc2 h3")

        # 將日期和標題對應成字典，然後添加到列表中
        for date_element, title_element in zip(date_elements, title_elements):
            data3 = {
                'date': date_element.text,
                'title': title_element.text
            }
            data_list3.append(data3)

        i=i+1
        if temp_div_3 == 7:
            temp_div_3 = 1
            if temp_div_2 == 7:
                temp_div_2 = 2
                if temp_div_1 == 2:
                    nextpage = driver.find_element(By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]')
                    nextpage.click()
                    temp_div_1 = 1
                else:
                    temp_div_1 = temp_div_1 + 1
            else:
                temp_div_2 = temp_div_2 + 1
        else:
            temp_div_3 = temp_div_3 + 1

        # 此輪迴圈結束後，將暫時的變數賦值給 div_1、div_2 和 div_3
        div_1 = temp_div_1
        div_2 = temp_div_2
        div_3 = temp_div_3
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
    except NoSuchElementException:
        if(div_3==7):
            div_3=1
            if(div_2==7):
                div_2=2
                if(div_1==2):
                    nextpage=driver.find_element(By.XPATH,'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]').click()
                    div_1=1
                else:
                    div_1=div_1+1
            else:
                div_2=div_2+1
        else:
            div_3=div_3+1
        date3=driver.find_element(By.CLASS_NAME,'_Qfx4')
        date3.click()
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
driver.close()
# 輸出所有日期和標題對應的資料
for data3 in data_list3:
    print(data3['date'], data3['title'])
            
with open('新聞標題(2022~2023)/news_3.csv', mode='w', encoding='utf-8', newline='') as csvfile:
    # 定義欄位
    fieldnames = ['date', 'title']

    # 將 dictionary 寫入 CSV 檔
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # 寫入第一列的欄位名稱
    writer.writeheader()
    for data3 in data_list3:
        # 寫入資料
        writer.writerow({'date': data3['date'], 'title': data3['title']})

223
224
225
226
227
231
232
233
234
235
236
237
241
242
243
244
245
246
247
251
252
253
254
255
256
257
261
262
263
264
265
2022/03/01 刺激投保意願 五張美元利變保單3月調升利率
2022/03/01 興采、聚紡股臨會通過合併案 在台擴產強化整合綜效
2022/03/01 禾昌2月營收雙減 累計營收力守持平去年
2022/03/01 東研信超取得TAF認可 將強化WiFi 6E檢測能力
2022/03/01 〈宏達電展MWC〉不放棄手機市場 預計4月發表元宇宙新機
2022/03/01 從主動式智慧防災出發 宇瞻首重為客戶創造價值
2022/03/01 網龍挾IP優勢大啖元宇宙商機 今年將推3款NFT
2022/03/01 畢業生最嚮往公司！富邦金控集團12度蟬聯「保險龍鳳獎」四項特優
2022/03/01 大宇紡織虧損收斂 楊任凱出任總座
2022/03/01 泰鼎-KY去年獲利創高 EPS 7.25元 擴產效益逐步顯現
2022/03/01 聚積攜手洲明 搶進LED電影屏市場
2022/03/01 真好玩4月推自研手遊《吞食天地3：放置版》 今年營運可望轉盈
2022/03/01 安碁資訊去年EPS 5.11元 每股擬配3.7元現金股利
2022/03/01 俄羅斯被逐出SWIFT 四大銀行、三大壽險踩雷
2022/03/01 遠傳併購亞太電首年即見綜效 EBITDA至少增30億元
2022/03/01 力麗轉型有成 今年月均營收估站穩10億元 全年成長逾1成
2022/03/01 台積電日本熊本廠釋最新進度 預計Q2動工
2022/03/01 〈竹科跳電〉友達部分機台異常持續復機中 群創未受影響
2022/03/01 〈車市〉電動機車買氣旺 前2月掛牌數年增逾130%
2022/03/01 〈高端疫苗董事會〉因應未來國際訂單需求提前備料 擬增資募35億元
2022/03/01 〈竹科跳電〉台積電確認影響中 力積電仍在查明原因
2022/03/01 鋼鐵族群為什麼這麼強？除了鋼鐵以外，還有這兩大族群也是趨勢
2022/03/01 自行車整車廠明係去年獲利大增91.7% 每股純益1.85元
2022/03/01 〈竹科跳電〉台電證實

In [24]:
#4月
driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
data_list4 = []
data4={}#使用大刮號{}創建字典
url='https://news.cnyes.com/news/cat/tw_stock_news?exp=a'
driver.get(url)

date=driver.find_element(By.CLASS_NAME,'_Qfx4')
date.click()
div_1=2 #1or2
div_2=2 #2~7
div_3=6 #1~7
i=0;
#日期往前按17頁
for i in range(17):
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[1]/div[1]/div[1]'))).click()
#4月
for i in range(30):
    try:
        # 初始化列表用於存儲日期和標題
        data_list = []
        print(f"{div_1}{div_2}{div_3}")
        if(i!=0):
            # 使用 JavaScript 進行滾動
            driver.execute_script("window.scrollBy(0, 300);")
            # 選擇日期選擇器
            date_selector = driver.find_element(By.CLASS_NAME, '_Qfx4')
            date_selector.click()
        # 暫時的變數
        temp_div_1 = div_1
        temp_div_2 = div_2
        temp_div_3 = div_3

        # 選擇4/1    
        start = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(start))
        start2 = driver.find_element(*start)
        action = ActionChains(driver)
        action.move_to_element(start2).click().perform()
        # 選擇4/1
        end = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(end))
        end2 = driver.find_element(*end)
        action2 = ActionChains(driver)
        action2.move_to_element(end2).click().perform()
        #確定按鈕
        button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[2]/button[1]')
        driver.execute_script("arguments[0].click();", button)

        time.sleep(5)

    
#模擬向下捲動，以確保載入更多內容
        scrolls = 3
        for _ in range(scrolls):
            driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
            time.sleep(2)  # 等待頁面載入
#找到當前頁面
        page_content=driver.page_source

# 使用 BeautifulSoup 解析頁面内容
        soup = BeautifulSoup(page_content, "html.parser")

        # 抓取日期元素
        date_elements = soup.select("div._67tN.theme-meta time")

        # 抓取標題元素
        title_elements = soup.select("div._1xc2 h3")

        # 將日期和標題對應成字典，然後添加到列表中
        for date_element, title_element in zip(date_elements, title_elements):
            data4 = {
                'date': date_element.text,
                'title': title_element.text
            }
            data_list4.append(data4)

        i=i+1
        if temp_div_3 == 7:
            temp_div_3 = 1
            if temp_div_2 == 7:
                temp_div_2 = 2
                if temp_div_1 == 2:
                    nextpage = driver.find_element(By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]')
                    nextpage.click()
                    temp_div_1 = 1
                else:
                    temp_div_1 = temp_div_1 + 1
            else:
                temp_div_2 = temp_div_2 + 1
        else:
            temp_div_3 = temp_div_3 + 1

        # 此輪迴圈結束後，將暫時的變數賦值給 div_1、div_2 和 div_3
        div_1 = temp_div_1
        div_2 = temp_div_2
        div_3 = temp_div_3
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
    except NoSuchElementException:
        if(div_3==7):
            div_3=1
            if(div_2==7):
                div_2=2
                if(div_1==2):
                    nextpage=driver.find_element(By.XPATH,'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]').click()
                    div_1=1
                else:
                    div_1=div_1+1
            else:
                div_2=div_2+1
        else:
            div_3=div_3+1
        date3=driver.find_element(By.CLASS_NAME,'_Qfx4')
        date3.click()
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
driver.close()
# 輸出所有日期和標題對應的資料
for data4 in data_list4:
    print(data4['date'], data4['title'])
            
with open('新聞標題(2022~2023)/news_4.csv', mode='w', encoding='utf-8', newline='') as csvfile:
    # 定義欄位
    fieldnames = ['date', 'title']

    # 將 dictionary 寫入 CSV 檔
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # 寫入第一列的欄位名稱
    writer.writeheader()
    for data4 in data_list4:
        # 寫入資料
        writer.writerow({'date': data4['date'], 'title': data4['title']})

226
227
231
232
233
234
235
236
237
241
242
243
244
245
246
247
251
252
253
254
255
256
257
261
262
263
264
265
266
267
2022/04/01 本土疫情升溫 台積電超前部署明起分組分流辦公
2022/04/01 台灣高鐵受外力影響致營運中斷 將研議求償
2022/04/01 承包商施工不慎撞倒電塔殃及高鐵 東泥：會負起監督責任
2022/04/01 證交所：48家上市公司買回庫藏股 過半執行率100%
2022/04/01 佳世達處分轉投資持股 貢獻EPS 2.7元
2022/04/01 顯卡市場競爭恐更激烈 摩爾線程正在崛起
2022/04/01 國巨出售日本仙台工廠 處分利益逾6.5億元
2022/04/01 台開等4家上市公司財報難產 7日起暫停交易
2022/04/01 南茂送暖至偏鄉孩童 認養146份兒童節禮物
2022/04/01 弘憶股今年車用產品業績數倍增 全年多產品續揚
2022/04/01 印度第二大電信商估5G O-RAN下半年商轉 台網通廠添動能
2022/04/01 智慧顯示展Touch Taiwan月底登場 台廠秀MicroLED肌肉
2022/04/01 原料價格單月飆漲逾3成 台橡Q2營運有壓
2022/04/01 〈錸寶展望〉Q2兩大事業動能回溫 營運將優於Q1
2022/04/01 台灣大商務長 前宏佳騰執行長林東閔出任
2022/04/01 鈞興-KY越南新廠擴大接單 Q3可望單月損平
2022/04/01 外資大舉提款晶圓雙雄近80億元 續加碼金融股
2022/04/01 先別管清明節變盤不變盤 四月份不可輕忽潛力股
2022/04/01 〈錸寶展望〉再斥1.7億元加碼投資達振能源 持股比達50.12%
2022/04/01 美琪瑪擴充回收、高附加價值產線 今年獲利結構再優化
2022/04/01 〈兒童理財〉子女保障信託正夯 簽約注意兩眉角
2022/04/01 約定條件無法達成 台達電出售加拿大商設備案終止
2022/04/01 健身需求成長+台幣走穩 岱宇今年毛利率優於去年
2022/04/01 節前量縮不用擔心，「這些」族群節後將成為盤面焦點
2022/04/01 聯茂攜三菱瓦斯化學設立合資公司 切入載

In [25]:
#5月
driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
data_list5 = []
data5={}#使用大刮號{}創建字典
url='https://news.cnyes.com/news/cat/tw_stock_news?exp=a'
driver.get(url)

date=driver.find_element(By.CLASS_NAME,'_Qfx4')
date.click()
div_1=2 #1or2
div_2=2 #2~7
div_3=1 #1~7
i=0;
#日期往前按16頁
for i in range(16):
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[1]/div[1]/div[1]'))).click()
#5月
for i in range(31):
    try:
        # 初始化列表用於存儲日期和標題
        data_list = []
        print(f"{div_1}{div_2}{div_3}")
        if(i!=0):
            # 使用 JavaScript 進行滾動
            driver.execute_script("window.scrollBy(0, 300);")
            # 選擇日期選擇器
            date_selector = driver.find_element(By.CLASS_NAME, '_Qfx4')
            date_selector.click()
        # 暫時的變數
        temp_div_1 = div_1
        temp_div_2 = div_2
        temp_div_3 = div_3

        # 選擇3/1    
        start = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(start))
        start2 = driver.find_element(*start)
        action = ActionChains(driver)
        action.move_to_element(start2).click().perform()
        # 選擇3/1
        end = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(end))
        end2 = driver.find_element(*end)
        action2 = ActionChains(driver)
        action2.move_to_element(end2).click().perform()
        #確定按鈕
        button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[2]/button[1]')
        driver.execute_script("arguments[0].click();", button)

        time.sleep(5)

    
#模擬向下捲動，以確保載入更多內容
        scrolls = 3
        for _ in range(scrolls):
            driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
            time.sleep(2)  # 等待頁面載入
#找到當前頁面
        page_content=driver.page_source

# 使用 BeautifulSoup 解析頁面内容
        soup = BeautifulSoup(page_content, "html.parser")

        # 抓取日期元素
        date_elements = soup.select("div._67tN.theme-meta time")

        # 抓取標題元素
        title_elements = soup.select("div._1xc2 h3")

        # 將日期和標題對應成字典，然後添加到列表中
        for date_element, title_element in zip(date_elements, title_elements):
            data5 = {
                'date': date_element.text,
                'title': title_element.text
            }
            data_list5.append(data5)

        i=i+1
        if temp_div_3 == 7:
            temp_div_3 = 1
            if temp_div_2 == 7:
                temp_div_2 = 2
                if temp_div_1 == 2:
                    nextpage = driver.find_element(By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]')
                    nextpage.click()
                    temp_div_1 = 1
                else:
                    temp_div_1 = temp_div_1 + 1
            else:
                temp_div_2 = temp_div_2 + 1
        else:
            temp_div_3 = temp_div_3 + 1

        # 此輪迴圈結束後，將暫時的變數賦值給 div_1、div_2 和 div_3
        div_1 = temp_div_1
        div_2 = temp_div_2
        div_3 = temp_div_3
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
    except NoSuchElementException:
        if(div_3==7):
            div_3=1
            if(div_2==7):
                div_2=2
                if(div_1==2):
                    nextpage=driver.find_element(By.XPATH,'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]').click()
                    div_1=1
                else:
                    div_1=div_1+1
            else:
                div_2=div_2+1
        else:
            div_3=div_3+1
        date3=driver.find_element(By.CLASS_NAME,'_Qfx4')
        date3.click()
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
driver.close()
# 輸出所有日期和標題對應的資料
for data5 in data_list5:
    print(data5['date'], data5['title'])
            
with open('新聞標題(2022~2023)/news_5.csv', mode='w', encoding='utf-8', newline='') as csvfile:
    # 定義欄位
    fieldnames = ['date', 'title']

    # 將 dictionary 寫入 CSV 檔
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # 寫入第一列的欄位名稱
    writer.writeheader()
    for data5 in data_list5:
        # 寫入資料
        writer.writerow({'date': data5['date'], 'title': data5['title']})

221
222
223
224
225
226
227
231
232
233
234
235
236
237
241
242
243
244
245
246
247
251
252
253
254
255
256
257
261
262
263
2022/05/01 科技大廠法說、4月CPI、綜所稅起跑 本周大事預告
2022/05/01 〈觀察〉工具機業訂單滿手苦於缺工 重啟移工助產能拉升
2022/05/01 〈觀察〉友達攻電子紙商機 推膽固醇液晶技術迎發展元年
2022/05/01 台股大幅修正 大盤何時反彈看兩大因素
2022/05/01 〈觀察〉防疫保單絕跡背後 隱藏產險業增資潮危機
2022/05/02 紅單交易如改課房地合一稅 財部允諾給輔導期補申報
2022/05/02 歐盟對中國石墨電極棒下重手課稅最多75% 中碳啖轉單商機
2022/05/02 〈金管會備詢〉確診5月起轉向自主回報 保險局指理賠看一關鍵
2022/05/02 5月報稅季開跑 逾26萬戶快手完成申報 4成用手機
2022/05/02 勞動基金3月收益回升948億元 首季虧損縮小
2022/05/02 〈金管會備詢〉防疫理賠大增 產險業有481億元準備金可沖抵
2022/05/02 華邦電20奈米製程開發順利 搶元宇宙應用商機
2022/05/03 宏碁旗下三金雞Q1報喜 安碁資訊改寫單季新高
2022/05/03 富鼎Q1毛利率衝38%創新高 EPS 3.53元超越去年上半年
2022/05/03 永豐餘投控榮獲遠見CSR暨ESG企業社會責任獎肯定
2022/05/03 料況改善 融程電首季EPS 1.19元寫同期高 醫揚Q1純益年增1.4倍
2022/05/03 明基材擬配1.5元現金股利 Q1每股賺0.91元近八年新高
2022/05/03 正基競拍底價83.48元 預定5月24日掛牌
2022/05/03 新唐Q1純益年增344% 每股賺2.87元同步攀峰
2022/05/03 沃旭離岸風場陸上變電站啟用 併聯規模全台最大
2022/05/03 泰博Q1賺逾半股本創次高 EPS 6.69元
2022/05/03 Tablet, Chromebook Shipments Decline in 1st Quarter as Peak Demand Passed
2022/05/

In [27]:
#6月
driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
data_list6 = []
data6={}#使用大刮號{}創建字典
url='https://news.cnyes.com/news/cat/tw_stock_news?exp=a'
driver.get(url)

date=driver.find_element(By.CLASS_NAME,'_Qfx4')
date.click()
div_1=2 #1or2
div_2=2 #2~7
div_3=4 #1~7
i=0;
#日期往前按15頁
for i in range(15):
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[1]/div[1]/div[1]'))).click()
#6月
for i in range(30):
    try:
        # 初始化列表用於存儲日期和標題
        data_list = []
        print(f"{div_1}{div_2}{div_3}")
        if(i!=0):
            # 使用 JavaScript 進行滾動
            driver.execute_script("window.scrollBy(0, 300);")
            # 選擇日期選擇器
            date_selector = driver.find_element(By.CLASS_NAME, '_Qfx4')
            date_selector.click()
        # 暫時的變數
        temp_div_1 = div_1
        temp_div_2 = div_2
        temp_div_3 = div_3

        # 選擇6/1    
        start = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(start))
        start2 = driver.find_element(*start)
        action = ActionChains(driver)
        action.move_to_element(start2).click().perform()
        # 選擇6/1
        end =(By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(end))
        end2 = driver.find_element(*end)
        action2 = ActionChains(driver)
        action2.move_to_element(end2).click().perform()
        #確定按鈕
        button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[2]/button[1]')
        driver.execute_script("arguments[0].click();", button)

        time.sleep(5)

    
#模擬向下捲動，以確保載入更多內容
        scrolls = 3
        for _ in range(scrolls):
            driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
            time.sleep(2)  # 等待頁面載入
#找到當前頁面
        page_content=driver.page_source

# 使用 BeautifulSoup 解析頁面内容
        soup = BeautifulSoup(page_content, "html.parser")

        # 抓取日期元素
        date_elements = soup.select("div._67tN.theme-meta time")

        # 抓取標題元素
        title_elements = soup.select("div._1xc2 h3")

        # 將日期和標題對應成字典，然後添加到列表中
        for date_element, title_element in zip(date_elements, title_elements):
            data6 = {
                'date': date_element.text,
                'title': title_element.text
            }
            data_list6.append(data6)

        i=i+1
        if temp_div_3 == 7:
            temp_div_3 = 1
            if temp_div_2 == 7:
                temp_div_2 = 2
                if temp_div_1 == 2:
                    nextpage = driver.find_element(By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]')
                    nextpage.click()
                    temp_div_1 = 1
                else:
                    temp_div_1 = temp_div_1 + 1
            else:
                temp_div_2 = temp_div_2 + 1
        else:
            temp_div_3 = temp_div_3 + 1

        # 此輪迴圈結束後，將暫時的變數賦值給 div_1、div_2 和 div_3
        div_1 = temp_div_1
        div_2 = temp_div_2
        div_3 = temp_div_3
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
    except NoSuchElementException:
        if(div_3==7):
            div_3=1
            if(div_2==7):
                div_2=2
                if(div_1==2):
                    nextpage=driver.find_element(By.XPATH,'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]').click()
                    div_1=1
                else:
                    div_1=div_1+1
            else:
                div_2=div_2+1
        else:
            div_3=div_3+1
        date3=driver.find_element(By.CLASS_NAME,'_Qfx4')
        date3.click()
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
driver.close()
# 輸出所有日期和標題對應的資料
for data6 in data_list6:
    print(data6['date'], data6['title'])
            
with open('新聞標題(2022~2023)/news_6.csv', mode='w', encoding='utf-8', newline='') as csvfile:
    # 定義欄位
    fieldnames = ['date', 'title']

    # 將 dictionary 寫入 CSV 檔
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # 寫入第一列的欄位名稱
    writer.writeheader()
    for data6 in data_list6:
        # 寫入資料
        writer.writerow({'date': data6['date'], 'title': data6['title']})

224
225
226
227
231
232
233
234
235
236
237
241
242
243
244
245
246
247
251
252
253
254
255
256
257
261
262
263
264
265
2022/06/01 欣陸投控旗下大陸建設今年推案估210億元 竹、桃尋找適合開發標的
2022/06/01 《非凡贏家》車用二極體蓄勢待發！
2022/06/01 〈日電貿法說〉鋁電、聚合物電容需求穩健向上 交期長達20周
2022/06/01 〈ESG大趨勢〉遠傳加入愛立信環保計畫 汰換上萬基站、回收98%電子廢棄物
2022/06/01 晟鈦董事會大改組 前大慶證董座莊明理出任董事長
2022/06/01 外資提款集中金融股 買超聚焦面板、航運股
2022/06/01 長聖5月、前5月營收雙創新猷 全年有望突破7億元
2022/06/01 〈旺宏展望〉吳敏求：車用NOR Flash報價已超越英飛凌
2022/06/01 保險存摺正式上線 年繳百元可不限次查所有保單
2022/06/01 〈旺宏展望〉吳敏求：旺宏股價一直是委屈
2022/06/01 旭源4月轉盈 前4月每股純益9.27元
2022/06/01 台塑化擬斥資3.42億元 採購廢熱處理等設備
2022/06/01 〈日電貿法說〉庫存去化估8月告一段落 下半年被動元件需求持續成長
2022/06/01 上海全面解封，電子族群機會來了？首要關注這兩大族群
2022/06/01 大魯閣休閒事業拚拓點 湳雅廣場將增營業面積
2022/06/01 〈開發金法說〉擬備妥400億銀彈 瞄準評估併購機會
2022/06/01 俄烏戰爭+中國封控 全球手機產量不排除再下修
2022/06/01 惠普看旺企業需求台廠營運有撐 聯想也推新品搶市
2022/06/01 〈車市〉三陽5月銷量衝破1.5萬台 重回銷售冠軍
2022/06/01 傳三安光電MiniLED產品驗證不順 最快下半年量產
2022/06/01 〈開發金法說〉中壽若接軌IFRS 17 淨值估增加750億元
2022/06/01 季線反壓真的過不去嗎?8478東哥、1605華新、6235華孚見好就收?！ .
2022/06/01 宏碁智醫海外布局報捷 AI軟體醫材導入泰國、印度醫院
2022/06/01 〈車市〉缺車衝擊

In [28]:
#7月
driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
data_list7 = []
data7={}#使用大刮號{}創建字典
url='https://news.cnyes.com/news/cat/tw_stock_news?exp=a'
driver.get(url)

date=driver.find_element(By.CLASS_NAME,'_Qfx4')
date.click()
div_1=2 #1or2
div_2=2 #2~7
div_3=6 #1~7
i=0;
#日期往前按14頁
for i in range(14):
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[1]/div[1]/div[1]'))).click()
#7月
for i in range(31):
    try:
        # 初始化列表用於存儲日期和標題
        data_list = []
        print(f"{div_1}{div_2}{div_3}")
        if(i!=0):
            # 使用 JavaScript 進行滾動
            driver.execute_script("window.scrollBy(0, 300);")
            # 選擇日期選擇器
            date_selector = driver.find_element(By.CLASS_NAME, '_Qfx4')
            date_selector.click()
        # 暫時的變數
        temp_div_1 = div_1
        temp_div_2 = div_2
        temp_div_3 = div_3

        # 選擇7/1    
        start = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(start))
        start2 = driver.find_element(*start)
        action = ActionChains(driver)
        action.move_to_element(start2).click().perform()
        # 選擇7/1
        end = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(end))
        end2 = driver.find_element(*end)
        action2 = ActionChains(driver)
        action2.move_to_element(end2).click().perform()
        #確定按鈕
        button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[2]/button[1]')
        driver.execute_script("arguments[0].click();", button)

        time.sleep(5)

    
#模擬向下捲動，以確保載入更多內容
        scrolls = 3
        for _ in range(scrolls):
            driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
            time.sleep(2)  # 等待頁面載入
#找到當前頁面
        page_content=driver.page_source

# 使用 BeautifulSoup 解析頁面内容
        soup = BeautifulSoup(page_content, "html.parser")

        # 抓取日期元素
        date_elements = soup.select("div._67tN.theme-meta time")

        # 抓取標題元素
        title_elements = soup.select("div._1xc2 h3")

        # 將日期和標題對應成字典，然後添加到列表中
        for date_element, title_element in zip(date_elements, title_elements):
            data7 = {
                'date': date_element.text,
                'title': title_element.text
            }
            data_list7.append(data7)
            
        i=i+1
        if temp_div_3 == 7:
            temp_div_3 = 1
            if temp_div_2 == 7:
                temp_div_2 = 2
                if temp_div_1 == 2:
                    nextpage = driver.find_element(By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]')
                    nextpage.click()
                    temp_div_1 = 1
                else:
                    temp_div_1 = temp_div_1 + 1
            else:
                temp_div_2 = temp_div_2 + 1
        else:
            temp_div_3 = temp_div_3 + 1

        # 此輪迴圈結束後，將暫時的變數賦值給 div_1、div_2 和 div_3
        div_1 = temp_div_1
        div_2 = temp_div_2
        div_3 = temp_div_3
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
    except NoSuchElementException:
        if(div_3==7):
            div_3=1
            if(div_2==7):
                div_2=2
                if(div_1==2):
                    nextpage=driver.find_element(By.XPATH,'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]').click()
                    div_1=1
                else:
                    div_1=div_1+1
            else:
                div_2=div_2+1
        else:
            div_3=div_3+1
        date3=driver.find_element(By.CLASS_NAME,'_Qfx4')
        date3.click()
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
driver.close()
# 輸出所有日期和標題對應的資料
for data7 in data_list7:
    print(data7['date'], data7['title'])
            
with open('新聞標題(2022~2023)/news_7.csv', mode='w', encoding='utf-8', newline='') as csvfile:
    # 定義欄位
    fieldnames = ['date', 'title']

    # 將 dictionary 寫入 CSV 檔
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # 寫入第一列的欄位名稱
    writer.writeheader()
    for data7 in data_list7:
        # 寫入資料
        writer.writerow({'date': data7['date'], 'title': data7['title']})

226
227
231
232
233
234
235
236
237
241
242
243
244
245
246
247
251
252
253
254
255
256
257
261
262
263
264
265
266
267
271
2022/07/01 經濟成長挹注稅收 今年綜所稅申報額2259億元 年增逾2成
2022/07/01 台股市值單周蒸發2.8兆元 短線觀察三訊號變化
2022/07/01 德微6月營收1.98億元 Q2、上半年全創新高
2022/07/01 台開掰了！證交所公告8/4將強制下市
2022/07/01 資金出走潮 台股今年千金俱樂部成員腰斬 市值蒸發1.92兆元
2022/07/01 基泰國產實業大樓65億元易主 處分利益每股純益4.56元
2022/07/01 電動車電池大短缺 鈣鈦礦太陽能車頂是解藥
2022/07/01 0050下半年配息出爐 創同期最高金額
2022/07/01 鴻海、國泰金等26檔股票7/4除息交易 大盤估約蒸發近50點
2022/07/01 仲琦6月營收13.75億元創新高 下半年逐季增溫
2022/07/01 新盛力前5月獲利大增 EPS 0.93元轉盈
2022/07/01 台股熊市氛圍瀰漫 沈榮津：必要時將開國安基金臨時委員會
2022/07/01 〈車市〉缺車問題未解 6月車商排名再洗牌 BMW罕見衝上第二
2022/07/01 外資回補ABF族群 大買欣興 棄息鴻海賣超1.75萬張
2022/07/01 〈航運指數〉海運Q3旺季有雜音？ SCFI連3跌 三大長程線運價創波段低
2022/07/01 綠色金融助攻 北富銀、友達攜手全台首件永續供應鏈金融合作
2022/07/01 小心被假期交所網站誘騙開戶下單 一招教你識破騙局
2022/07/01 台泥總經理李鐘培辭職 張安平暫兼總座
2022/07/01 泰碩實施庫藏股 擬買回600張轉讓予員工
2022/07/01 市況變化 外資調降巨大評等至中立目標價砍到250元
2022/07/01 反映升息 2家壽險270張保單利率喊漲 台新壽不升反降
2022/07/01 圓剛攜手圓展 助力培育台灣空手道國手
2022/07/01 友達攜手供應鏈夥伴 目標2030年減碳20%
2022/07/01 浩鼎抗癌抗體新藥OBI-888臨床提前終止

In [29]:
#8月
driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
data_list8 = []
data8={}#使用大刮號{}創建字典
url='https://news.cnyes.com/news/cat/tw_stock_news?exp=a'
driver.get(url)

date=driver.find_element(By.CLASS_NAME,'_Qfx4')
date.click()
div_1=2 #1or2
div_2=2 #2~7
div_3=2 #1~7
i=0;
#日期往前按13頁
for i in range(13):
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[1]/div[1]/div[1]'))).click()
#8月
for i in range(31):
    try:
        # 初始化列表用於存儲日期和標題
        data_list = []
        print(f"{div_1}{div_2}{div_3}")
        if(i!=0):
            # 使用 JavaScript 進行滾動
            driver.execute_script("window.scrollBy(0, 300);")
            # 選擇日期選擇器
            date_selector = driver.find_element(By.CLASS_NAME, '_Qfx4')
            date_selector.click()
        # 暫時的變數
        temp_div_1 = div_1
        temp_div_2 = div_2
        temp_div_3 = div_3

        # 選擇8/1    
        start = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(start))
        start2 = driver.find_element(*start)
        action = ActionChains(driver)
        action.move_to_element(start2).click().perform()
        # 選擇8/1
        end = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(end))
        end2 = driver.find_element(*end)
        action2 = ActionChains(driver)
        action2.move_to_element(end2).click().perform()
        #確定按鈕
        button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[2]/button[1]')
        driver.execute_script("arguments[0].click();", button)

        time.sleep(5)

    
#模擬向下捲動，以確保載入更多內容
        scrolls = 3
        for _ in range(scrolls):
            driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
            time.sleep(2)  # 等待頁面載入
#找到當前頁面
        page_content=driver.page_source

# 使用 BeautifulSoup 解析頁面内容
        soup = BeautifulSoup(page_content, "html.parser")

        # 抓取日期元素
        date_elements = soup.select("div._67tN.theme-meta time")

        # 抓取標題元素
        title_elements = soup.select("div._1xc2 h3")

        # 將日期和標題對應成字典，然後添加到列表中
        for date_element, title_element in zip(date_elements, title_elements):
            data8 = {
                'date': date_element.text,
                'title': title_element.text
            }
            data_list8.append(data8)
            
        i=i+1
        if temp_div_3 == 7:
            temp_div_3 = 1
            if temp_div_2 == 7:
                temp_div_2 = 2
                if temp_div_1 == 2:
                    nextpage = driver.find_element(By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]')
                    nextpage.click()
                    temp_div_1 = 1
                else:
                    temp_div_1 = temp_div_1 + 1
            else:
                temp_div_2 = temp_div_2 + 1
        else:
            temp_div_3 = temp_div_3 + 1

        # 此輪迴圈結束後，將暫時的變數賦值給 div_1、div_2 和 div_3
        div_1 = temp_div_1
        div_2 = temp_div_2
        div_3 = temp_div_3
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
    except NoSuchElementException:
        if(div_3==7):
            div_3=1
            if(div_2==7):
                div_2=2
                if(div_1==2):
                    nextpage=driver.find_element(By.XPATH,'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]').click()
                    div_1=1
                else:
                    div_1=div_1+1
            else:
                div_2=div_2+1
        else:
            div_3=div_3+1
        date3=driver.find_element(By.CLASS_NAME,'_Qfx4')
        date3.click()
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
driver.close()
# 輸出所有日期和標題對應的資料
for data8 in data_list8:
    print(data8['date'], data8['title'])
            
with open('新聞標題(2022~2023)/news_8.csv', mode='w', encoding='utf-8', newline='') as csvfile:
    # 定義欄位
    fieldnames = ['date', 'title']

    # 將 dictionary 寫入 CSV 檔
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # 寫入第一列的欄位名稱
    writer.writeheader()
    for data8 in data_list8:
        # 寫入資料
        writer.writerow({'date': data8['date'], 'title': data8['title']})

222
223
224
225
226
227
231
232
233
234
235
236
237
241
242
243
244
245
246
247
251
252
253
254
255
256
257
261
262
263
264
2022/08/01 台股萬五得而復失 法人估基本面撐腰短線將反覆打底
2022/08/01 和鑫Q2每股賺0.12元 擬買回2萬張庫藏股
2022/08/01 台汽電擬斥7億元 在雲林建置儲能系統
2022/08/01 驅動IC庫存調整壓力大 外資估頎邦Q3營收季減2-3成
2022/08/01 智聯服務Q2純益創新高EPS 0.76元 上半年每股賺1.35元
2022/08/01 萬五關卡重重 生技類股擇優佈局
2022/08/01 明泰Q2每股賺0.45元 創5年來新高
2022/08/01 元宇宙、網通、下半年焦點股威盛、宏達電、兆赫
2022/08/01 〈ETF除息旺季〉國泰投信三檔ETF配息出爐 00878年化殖利率逾6% 
2022/08/01 台灣首家！中國信託勇奪《The Banker》最佳創新數位銀行大獎
2022/08/01 男性平均所得65萬高於女性 兩性差距縮小至1.5倍近10年首見
2022/08/01 泓格Q2純益季增9成寫次高 上半年EPS 1.84元同期最佳
2022/08/01 外資出脫台積電、三檔金融股 續回補新光金
2022/08/01 邁達特雲端服務事業擴大 下半年營運逐季增
2022/08/01 長聖前7月營收3.33億元續創高 Q4再走揚
2022/08/01 〈東洋展望〉複製台灣模式拓展東南亞市場 海外營收比重拚逾10%
2022/08/01 金麗科7月營收0.3億元 累計前7月年減29%
2022/08/01 中碳Q2純益近倍增創7年新高 EPS 2.41元
2022/08/01 伊雲谷看好下半年雲端動能 9月中旬轉上市
2022/08/01 顯卡價量齊跌 麗臺Q2轉虧EPS -1.25元、上半年每股虧0.78元
2022/08/01 〈ETF除息旺季〉00900擬配息1.2元 殖利率超過10%
2022/08/01 全家打造零售業首見電動車商店 南科試營運
2022/08/01 柏承庫藏股執行屆滿 共買回669張執行率66.9%
2022/08/01 萬五震盪，分析師：

In [30]:
#9月
driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
data_list9 = []
data9={}#使用大刮號{}創建字典
url='https://news.cnyes.com/news/cat/tw_stock_news?exp=a'
driver.get(url)

date=driver.find_element(By.CLASS_NAME,'_Qfx4')
date.click()
div_1=2 #1or2
div_2=2 #2~7
div_3=5 #1~7
i=0;
#日期往前按12頁
for i in range(12):
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[1]/div[1]/div[1]'))).click()
#9月
for i in range(30):
    try:
        # 初始化列表用於存儲日期和標題
        data_list = []
        print(f"{div_1}{div_2}{div_3}")
        if(i!=0):
            # 使用 JavaScript 進行滾動
            driver.execute_script("window.scrollBy(0, 300);")
            # 選擇日期選擇器
            date_selector = driver.find_element(By.CLASS_NAME, '_Qfx4')
            date_selector.click()
        # 暫時的變數
        temp_div_1 = div_1
        temp_div_2 = div_2
        temp_div_3 = div_3

        # 選擇9/1    
        start = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(start))
        start2 =driver.find_element(*start)
        action = ActionChains(driver)
        action.move_to_element(start2).click().perform()
        # 選擇9/1
        end = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(end))
        end2 = driver.find_element(*end)
        action2 = ActionChains(driver)
        action2.move_to_element(end2).click().perform()
        #確定按鈕
        button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[2]/button[1]')
        driver.execute_script("arguments[0].click();", button)

        time.sleep(5)

    
#模擬向下捲動，以確保載入更多內容
        scrolls = 3
        for _ in range(scrolls):
            driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
            time.sleep(2)  # 等待頁面載入
#找到當前頁面
        page_content=driver.page_source

# 使用 BeautifulSoup 解析頁面内容
        soup = BeautifulSoup(page_content, "html.parser")

        # 抓取日期元素
        date_elements = soup.select("div._67tN.theme-meta time")

        # 抓取標題元素
        title_elements = soup.select("div._1xc2 h3")

        # 將日期和標題對應成字典，然後添加到列表中
        for date_element, title_element in zip(date_elements, title_elements):
            data9 = {
                'date': date_element.text,
                'title': title_element.text
            }
            data_list9.append(data9)

        i=i+1
        if temp_div_3 == 7:
            temp_div_3 = 1
            if temp_div_2 == 7:
                temp_div_2 = 2
                if temp_div_1 == 2:
                    nextpage = driver.find_element(By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]')
                    nextpage.click()
                    temp_div_1 = 1
                else:
                    temp_div_1 = temp_div_1 + 1
            else:
                temp_div_2 = temp_div_2 + 1
        else:
            temp_div_3 = temp_div_3 + 1

        # 此輪迴圈結束後，將暫時的變數賦值給 div_1、div_2 和 div_3
        div_1 = temp_div_1
        div_2 = temp_div_2
        div_3 = temp_div_3
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
    except NoSuchElementException:
        if(div_3==7):
            div_3=1
            if(div_2==7):
                div_2=2
                if(div_1==2):
                    nextpage=driver.find_element(By.XPATH,'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]').click()
                    div_1=1
                else:
                    div_1=div_1+1
            else:
                div_2=div_2+1
        else:
            div_3=div_3+1
        date3=driver.find_element(By.CLASS_NAME,'_Qfx4')
        date3.click()
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
driver.close()
# 輸出所有日期和標題對應的資料
for data9 in data_list9:
    print(data9['date'], data9['title'])
            
with open('新聞標題(2022~2023)/news_9.csv', mode='w', encoding='utf-8', newline='') as csvfile:
    # 定義欄位
    fieldnames = ['date', 'title']

    # 將 dictionary 寫入 CSV 檔
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # 寫入第一列的欄位名稱
    writer.writeheader()
    for data9 in data_list9:
        # 寫入資料
        writer.writerow({'date': data9['date'], 'title': data9['title']})

225
226
227
231
232
233
234
235
236
237
241
242
243
244
245
246
247
251
252
253
254
255
256
257
261
262
263
264
265
266
2022/09/01 限電措施解除 亞泥四川子公司全面復工正常生產
2022/09/01 〈大同高層異動〉王金來接總座強調溝通關鍵 這一屆董事任期會努力做好
2022/09/01 〈大同高層異動〉華映破產 年底前財報除列可望認列76億元迴轉利益
2022/09/01 萬泰科全年營收衝百億 低軌衛星出貨增6倍 明年再翻倍
2022/09/01 土銀主辦中租迪和、合迪124億元聯貸案 銀行團參貸踴躍
2022/09/01 〈兆豐金法說〉搭升息順風車 四大策略提高財務操作收益
2022/09/01 〈兆豐金法說〉本土疫情升溫產險理賠壓力山大 兆豐產估Q4完成增資
2022/09/01 期交所訂「3+E」發展重點 客製化契約交易平台明年底前上線
2022/09/01 高端疫苗次世代新冠疫苗一期結果出爐 追加劑可對抗BA.4、BA.5
2022/09/01 〈車市〉三陽8月熱銷2.48萬台 年增逾35% 連4月奪銷售冠軍
2022/09/01 外資大舉提款台積電及金融股 逆勢加碼面板雙虎
2022/09/01 〈大同高層異動〉王光祥三點聲明 重申會在最快時間實現配股目標
2022/09/01 保瑞完成併購安成藥業 晉升國內產能最大藥業集團
2022/09/01 〈碩天法說〉東南亞產能提升減輕銷美關稅壓力 菲律賓廠續擴產
2022/09/01 〈大同高層異動〉副董王金來真除兼任總座 打造企業控股大艦隊
2022/09/01 老牌光學元件廠保勝光學 通過上櫃申請
2022/09/01 內需動能不熱 嘉里大榮：中秋檔期貨量年減約1成
2022/09/01 〈碩天法說〉下半年營運向上 訂單能見度達明年上半年
2022/09/01 生技廠報捷 東洋認列逾800萬元授權金 美時血癌藥獲美FDA核可
2022/09/01 三萬名股東注意！00713首次季配息 第一階段評價結果出爐
2022/09/01 原廠庫存壓力沉重 NAND Flash晶圓本季合約價跌幅恐擴大至35%
2022/09/01 高力前7月獲利年增47% 每股賺1.61元
2022/09/

In [31]:
#10月
driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
data_list10 = []
data10={}#使用大刮號{}創建字典
url='https://news.cnyes.com/news/cat/tw_stock_news?exp=a'
driver.get(url)

date=driver.find_element(By.CLASS_NAME,'_Qfx4')
date.click()
div_1=2 #1or2
div_2=2 #2~7
div_3=7 #1~7
i=0;
#日期往前按11頁
for i in range(11):
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[1]/div[1]/div[1]'))).click()
#10月
for i in range(31):
    try:
        # 初始化列表用於存儲日期和標題
        data_list = []
        print(f"{div_1}{div_2}{div_3}")
        if(i!=0):
            # 使用 JavaScript 進行滾動
            driver.execute_script("window.scrollBy(0, 300);")
            # 選擇日期選擇器
            date_selector = driver.find_element(By.CLASS_NAME, '_Qfx4')
            date_selector.click()
        # 暫時的變數
        temp_div_1 = div_1
        temp_div_2 = div_2
        temp_div_3 = div_3

        # 選擇10/1    
        start = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(start))
        start2 = driver.find_element(*start)
        action = ActionChains(driver)
        action.move_to_element(start2).click().perform()
        # 選擇10/1
        end = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(end))
        end2 = driver.find_element(*end)
        action2 = ActionChains(driver)
        action2.move_to_element(end2).click().perform()
        #確定按鈕
        button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[2]/button[1]')
        driver.execute_script("arguments[0].click();", button)

        time.sleep(5)

    
#模擬向下捲動，以確保載入更多內容
        scrolls = 3
        for _ in range(scrolls):
            driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
            time.sleep(2)  # 等待頁面載入
#找到當前頁面
        page_content=driver.page_source

# 使用 BeautifulSoup 解析頁面内容
        soup = BeautifulSoup(page_content, "html.parser")

        # 抓取日期元素
        date_elements = soup.select("div._67tN.theme-meta time")

        # 抓取標題元素
        title_elements = soup.select("div._1xc2 h3")

        # 將日期和標題對應成字典，然後添加到列表中
        for date_element, title_element in zip(date_elements, title_elements):
            data10 = {
                'date': date_element.text,
                'title': title_element.text
            }
            data_list10.append(data10)

        i=i+1
        if temp_div_3 == 7:
            temp_div_3 = 1
            if temp_div_2 == 7:
                temp_div_2 = 2
                if temp_div_1 == 2:
                    nextpage = driver.find_element(By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]')
                    nextpage.click()
                    temp_div_1 = 1
                else:
                    temp_div_1 = temp_div_1 + 1
            else:
                temp_div_2 = temp_div_2 + 1
        else:
            temp_div_3 = temp_div_3 + 1

        # 此輪迴圈結束後，將暫時的變數賦值給 div_1、div_2 和 div_3
        div_1 = temp_div_1
        div_2 = temp_div_2
        div_3 = temp_div_3
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
    except NoSuchElementException:
        if(div_3==7):
            div_3=1
            if(div_2==7):
                div_2=2
                if(div_1==2):
                    nextpage=driver.find_element(By.XPATH,'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]').click()
                    div_1=1
                else:
                    div_1=div_1+1
            else:
                div_2=div_2+1
        else:
            div_3=div_3+1
        date3=driver.find_element(By.CLASS_NAME,'_Qfx4')
        date3.click()
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
driver.close()
# 輸出所有日期和標題對應的資料
for data10 in data_list10:
    print(data10['date'], data10['title'])
            
with open('新聞標題(2022~2023)/news_10.csv', mode='w', encoding='utf-8', newline='') as csvfile:
    # 定義欄位
    fieldnames = ['date', 'title']

    # 將 dictionary 寫入 CSV 檔
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # 寫入第一列的欄位名稱
    writer.writeheader()
    for data10 in data_list10:
        # 寫入資料
        writer.writerow({'date': data10['date'], 'title': data10['title']})

227
231
232
233
234
235
236
237
241
242
243
244
245
246
247
251
252
253
254
255
256
257
261
262
263
264
265
266
267
271
272
2022/10/01 辜仲立餐飲新版圖問市 新品牌仲酣搶攻日本清酒商機
2022/10/01 為未來種一棵樹  中國信託同仁投入綠色行動
2022/10/01 中華電董座：未來電信商規模相當 市場競爭將更激烈
2022/10/01 〈熱門股〉Nike調庫存放緩拉貨 儒鴻失守400元寫2年低點
2022/10/01 〈熱門股〉資金撤出新藥族群 藥華藥破500元關卡 周跌16.6%
2022/10/01 〈熱門股〉奇鈦科本周重挫26% 跌破上櫃發行價
2022/10/01 〈熱門股〉嘉聯益Q3可望轉盈獲法人加碼 周漲逾10%
2022/10/01 〈熱門股〉宇峻Q4營運可望登峰 逆勢勁揚重回半年線
2022/10/01 台股匯雙殺、美光下修資本支出、台積電漲價喊卡 本周大事回顧
2022/10/01 〈熱門股〉建碁獲買盤追捧 單周大漲25%寫收盤新高價
2022/10/01 〈熱門股〉CIS市況遇亂流 精材周跌15%、創27個月新低
2022/10/01 〈研之有物〉無合成分裂：斑馬魚難以察覺的「表面功夫」
2022/10/02 茂迪TOPCON模組需求強 訂單滿至明年上半年
2022/10/02 中國風電回溫+歐美轉單挹注 上緯投控今年營運將優於去年
2022/10/02 〈觀察〉摩爾定律短短一周「死而復生」 各半導體廠爭論為哪樁？
2022/10/02 台廠顯卡的春天，三檔個股將綻放
2022/10/02 〈觀察〉中國自主品牌搶佔本地電動車市 日系合資廠加速追趕
2022/10/02 〈Q4投資策略〉10月聚焦三重點 不確定未除慎防台股下探萬三
2022/10/02 〈台股風向球〉總經利空烏雲罩頂 台股拚守萬三壓力大
2022/10/02 迅得估9月營收可望站穩5億元 Q3續改寫歷史新高
2022/10/02 〈Q4投資策略〉股債教戰 大型跌深股+高評等公債控制下檔風險
2022/10/02 上市櫃9月營收、國安基金例會、進出口統計 本周大事預告
2022/10/02 訊連旗下玩美移動赴美合併上市案 獲SEC審查生效
20

In [32]:
#11月
driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
data_list11 = []
data11={}#使用大刮號{}創建字典
url='https://news.cnyes.com/news/cat/tw_stock_news?exp=a'
driver.get(url)

date=driver.find_element(By.CLASS_NAME,'_Qfx4')
date.click()
div_1=2 #1or2
div_2=2 #2~7
div_3=3 #1~7
i=0;
#日期往前按10頁
for i in range(10):
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[1]/div[1]/div[1]'))).click()
#11月
for i in range(30):
    try:
        # 初始化列表用於存儲日期和標題
        data_list = []
        print(f"{div_1}{div_2}{div_3}")
        if(i!=0):
            # 使用 JavaScript 進行滾動
            driver.execute_script("window.scrollBy(0, 300);")
            # 選擇日期選擇器
            date_selector = driver.find_element(By.CLASS_NAME, '_Qfx4')
            date_selector.click()
        # 暫時的變數
        temp_div_1 = div_1
        temp_div_2 = div_2
        temp_div_3 = div_3

        # 選擇11/1    
        start = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(start))
        start2 = driver.find_element(*start)
        action = ActionChains(driver)
        action.move_to_element(start2).click().perform()
        # 選擇11/1
        end = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(end))
        end2 = driver.find_element(*end)
        action2 = ActionChains(driver)
        action2.move_to_element(end2).click().perform()
        #確定按鈕
        button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[2]/button[1]')
        driver.execute_script("arguments[0].click();", button)

        time.sleep(5)

    
#模擬向下捲動，以確保載入更多內容
        scrolls = 3
        for _ in range(scrolls):
            driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
            time.sleep(2)  # 等待頁面載入
#找到當前頁面
        page_content=driver.page_source

# 使用 BeautifulSoup 解析頁面内容
        soup = BeautifulSoup(page_content, "html.parser")

        # 抓取日期元素
        date_elements = soup.select("div._67tN.theme-meta time")

        # 抓取標題元素
        title_elements = soup.select("div._1xc2 h3")

        # 將日期和標題對應成字典，然後添加到列表中
        for date_element, title_element in zip(date_elements, title_elements):
            data11 = {
                'date': date_element.text,
                'title': title_element.text
            }
            data_list11.append(data11)

        i=i+1
        if temp_div_3 == 7:
            temp_div_3 = 1
            if temp_div_2 == 7:
                temp_div_2 = 2
                if temp_div_1 == 2:
                    nextpage = driver.find_element(By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]')
                    nextpage.click()
                    temp_div_1 = 1
                else:
                    temp_div_1 = temp_div_1 + 1
            else:
                temp_div_2 = temp_div_2 + 1
        else:
            temp_div_3 = temp_div_3 + 1

        # 此輪迴圈結束後，將暫時的變數賦值給 div_1、div_2 和 div_3
        div_1 = temp_div_1
        div_2 = temp_div_2
        div_3 = temp_div_3
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
    except NoSuchElementException:
        if(div_3==7):
            div_3=1
            if(div_2==7):
                div_2=2
                if(div_1==2):
                    nextpage=driver.find_element(By.XPATH,'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]').click()
                    div_1=1
                else:
                    div_1=div_1+1
            else:
                div_2=div_2+1
        else:
            div_3=div_3+1
        date3=driver.find_element(By.CLASS_NAME,'_Qfx4')
        date3.click()
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
driver.close()
# 輸出所有日期和標題對應的資料
for data11 in data_list11:
    print(data11['date'], data11['title'])
            
with open('新聞標題(2022~2023)/news_11.csv', mode='w', encoding='utf-8', newline='') as csvfile:
    # 定義欄位
    fieldnames = ['date', 'title']

    # 將 dictionary 寫入 CSV 檔
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # 寫入第一列的欄位名稱
    writer.writeheader()
    for data11 in data_list11:
        # 寫入資料
        writer.writerow({'date': data11['date'], 'title': data11['title']})

223
224
225
226
227
231
232
233
234
235
236
237
241
242
243
244
245
246
247
251
252
253
254
255
256
257
261
262
263
264
2022/11/01 明基材業外貢獻Q3每股賺1.67元創高 擬斥31億元取得衛普實業過半股權
2022/11/01 融程電Q3純益季增7成 EPS 2.24元締單季新猷
2022/11/01 〈中光電法說〉微投影機動能續強明年出貨看增6成 越南二廠明年Q3上線
2022/11/01 外資連二賣鴻海共2.2萬張 反手敲進面板雙虎4.66萬張
2022/11/01 帆宣Q3 EPS 4.23元創新高 前三季超越去年全年
2022/11/01 〈中光電法說〉Q4兩大產品出貨量估下滑 產業修正到明年Q1
2022/11/01 保勝光學前三季稅後純益8579萬元 每股純益3.07元
2022/11/01 PVC市況低迷 華夏前三季每股虧0.55元
2022/11/01 談美擴大對中晶片管制 愛普*看好有利加速3DIC發展
2022/11/01 閎康Q3稅後純益年增3成 EPS 2.97元創次高
2022/11/01 華新科Q3 EPS1.05元探21季低點 前三季純益年減近7成
2022/11/01 緯穎Q3登峰 前三季大賺百億並超越去年全年 EPS 58.48元
2022/11/01 三星目標明年擴大MicroLED電視陣容 供應鏈營運添動能
2022/11/01 〈環球晶法說〉8吋、12吋產能滿到明年Q1 德州新廠新長約入袋
2022/11/01 聯茂前三季稅後純益年減34% EPS 4.01元
2022/11/01 威盛Q3本業維持獲利 EPS 0.41元
2022/11/01 兩家產險公司申請純網保 預計明年4月放榜
2022/11/01 欣陸投控前三季稅後純益年增1.66倍 EPS 3.21元
2022/11/01 防疫雙險今年已理賠1441億元 產險業增資逾700億元
2022/11/01 矽格Q3 EPS 2.17元創次高 前三季逼近去年全年
2022/11/01 政府推低軌衛星PoC 中華電：願與任何業者合作
2022/11/01 柏承1000張庫藏股提前執行完畢 每股買回均價19.24元
2022/11/01 〈玖鼎登興

In [33]:
#12月
driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
data_list12 = []
data12={}#使用大刮號{}創建字典
url='https://news.cnyes.com/news/cat/tw_stock_news?exp=a'
driver.get(url)

date=driver.find_element(By.CLASS_NAME,'_Qfx4')
date.click()
div_1=2 #1or2
div_2=2 #2~7
div_3=5 #1~7
i=0;
#日期往前按9頁
for i in range(9):
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[1]/div[1]/div[1]'))).click()
#12月
for i in range(31):
    try:
        # 初始化列表用於存儲日期和標題
        data_list = []
        print(f"{div_1}{div_2}{div_3}")
        if(i!=0):
            # 使用 JavaScript 進行滾動
            driver.execute_script("window.scrollBy(0, 300);")
            # 選擇日期選擇器
            date_selector = driver.find_element(By.CLASS_NAME, '_Qfx4')
            date_selector.click()
        # 暫時的變數
        temp_div_1 = div_1
        temp_div_2 = div_2
        temp_div_3 = div_3

        # 選擇12/1    
        start = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(start))
        start2 = driver.find_element(*start)
        action = ActionChains(driver)
        action.move_to_element(start2).click().perform()
        # 選擇12/1
        end = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(end))
        end2 = driver.find_element(*end)
        action2 = ActionChains(driver)
        action2.move_to_element(end2).click().perform()
        #確定按鈕
        button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[2]/button[1]')
        driver.execute_script("arguments[0].click();", button)

        time.sleep(5)

    
#模擬向下捲動，以確保載入更多內容
        scrolls = 3
        for _ in range(scrolls):
            driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
            time.sleep(2)  # 等待頁面載入
#找到當前頁面
        page_content=driver.page_source

# 使用 BeautifulSoup 解析頁面内容
        soup = BeautifulSoup(page_content, "html.parser")

        # 抓取日期元素
        date_elements = soup.select("div._67tN.theme-meta time")

        # 抓取標題元素
        title_elements = soup.select("div._1xc2 h3")

        # 將日期和標題對應成字典，然後添加到列表中
        for date_element, title_element in zip(date_elements, title_elements):
            data12 = {
                'date': date_element.text,
                'title': title_element.text
            }
            data_list12.append(data12)

        i=i+1
        if temp_div_3 == 7:
            temp_div_3 = 1
            if temp_div_2 == 7:
                temp_div_2 = 2
                if temp_div_1 == 2:
                    nextpage = driver.find_element(By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]')
                    nextpage.click()
                    temp_div_1 = 1
                else:
                    temp_div_1 = temp_div_1 + 1
            else:
                temp_div_2 = temp_div_2 + 1
        else:
            temp_div_3 = temp_div_3 + 1

        # 此輪迴圈結束後，將暫時的變數賦值給 div_1、div_2 和 div_3
        div_1 = temp_div_1
        div_2 = temp_div_2
        div_3 = temp_div_3
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
    except NoSuchElementException:
        if(div_3==7):
            div_3=1
            if(div_2==7):
                div_2=2
                if(div_1==2):
                    nextpage=driver.find_element(By.XPATH,'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]').click()
                    div_1=1
                else:
                    div_1=div_1+1
            else:
                div_2=div_2+1
        else:
            div_3=div_3+1
        date3=driver.find_element(By.CLASS_NAME,'_Qfx4')
        date3.click()
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
driver.close()
# 輸出所有日期和標題對應的資料
for data12 in data_list12:
    print(data12['date'], data12['title'])
            
with open('新聞標題(2022~2023)/news_12.csv', mode='w', encoding='utf-8', newline='') as csvfile:
    # 定義欄位
    fieldnames = ['date', 'title']

    # 將 dictionary 寫入 CSV 檔
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # 寫入第一列的欄位名稱
    writer.writeheader()
    for data12 in data_list12:
        # 寫入資料
        writer.writerow({'date': data12['date'], 'title': data12['title']})

225
226
227
231
232
233
234
235
236
237
241
242
243
244
245
246
247
251
252
253
254
255
256
257
261
262
263
264
265
266
267
2022/12/01 金融業最嚴峻的一年 增資1995億元創史上最慘
2022/12/01 鑫龍騰布局北高雄開發案 總銷23億元岡山信義案動土
2022/12/01 世豐越南廠動土2024年Q1投產 年產值8億元
2022/12/01 〈醫療科技展〉鴻海發表健康雲平台「無疆雲」 積極孵育內部創業公司
2022/12/01 泓德能源斥資50.2億元 購置儲能設備
2022/12/01 〈中經院發布PMI〉我11月非製造業NMI轉為擴張 惟未來半年仍連8月緊縮
2022/12/01 松翰首推無線攝影機方案 獲大廠採用量產
2022/12/01 〈醫療科技展〉鴻海攜揚明光打造眼底影像解決方案 新品明年Q4上市
2022/12/01 〈中經院發布PMI〉我11月製造業PMI連5月緊縮 未來半年展望連7月緊縮
2022/12/01 新盛力10月獲利月減6成 前10月EPS 3.1元
2022/12/01 超幸運！麥當勞員工只花21元買員工餐就對中1000萬大獎
2022/12/01 六角明年總店數估淨增400家 設定美國市場為起飛元年
2022/12/01 生華科癌症新藥 獲選美國衛院五年共同開發計畫
2022/12/01 國泰世華銀行「又」當機 金管會罕見說重話 檢討層級拉高至董事會
2022/12/01 〈車市〉11月前10大車商僅5家正成長 賓士躍居銷售第二
2022/12/01 〈醫療科技展〉台塑環繞六大主題 秀大腸癌檢測等新品
2022/12/01 元富證舉辦投資論壇 前瞻2023生醫產業趨勢
2022/12/01 保勝光學完成上櫃前競拍 確定以每股32.8元IPO
2022/12/01 〈醫療科技展〉施崇棠：疫情重塑醫療樣貌 將建構智慧醫療虛實整合系統
2022/12/01 東森旗下熊媽媽買菜網將收攤 營業至12/8
2022/12/01 防疫措施鬆綁 6家產險防疫險理賠緊縮 確診須住院才能給付住院日額
2022/12/01 瑞耘第二階段擴產年底前完成 明年營收估再增10-15%
2022/12/01 誠品生活首度進軍東

In [5]:
#2023 1月
driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
data_list23_1 = []
data23_1={}#使用大刮號{}創建字典
url='https://news.cnyes.com/news/cat/tw_stock_news?exp=a'
driver.get(url)

date=driver.find_element(By.CLASS_NAME,'_Qfx4')
date.click()
div_1=2 #1or2
div_2=2 #2~7
div_3=1 #1~7
i=0;
#日期往前按9頁
for i in range(9):
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[1]/div[1]/div[1]'))).click()
#1月
for i in range(31):
    try:
        # 初始化列表用於存儲日期和標題
        data_list = []
        print(f"{div_1}{div_2}{div_3}")
        if(i!=0):
            # 使用 JavaScript 進行滾動
            driver.execute_script("window.scrollBy(0, 300);")
            # 選擇日期選擇器
            date_selector = driver.find_element(By.CLASS_NAME, '_Qfx4')
            date_selector.click()
        # 暫時的變數
        temp_div_1 = div_1
        temp_div_2 = div_2
        temp_div_3 = div_3

        # 選擇1/1    
        start = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(start))
        start2 = driver.find_element(*start)
        action = ActionChains(driver)
        action.move_to_element(start2).click().perform()
        # 選擇1/1
        end = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(end))
        end2 = driver.find_element(*end)
        action2 = ActionChains(driver)
        action2.move_to_element(end2).click().perform()
        #確定按鈕
        button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[2]/button[1]')
        driver.execute_script("arguments[0].click();", button)

        time.sleep(5)

    
#模擬向下捲動，以確保載入更多內容
        scrolls = 3
        for _ in range(scrolls):
            driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
            time.sleep(2)  # 等待頁面載入
#找到當前頁面
        page_content=driver.page_source

# 使用 BeautifulSoup 解析頁面内容
        soup = BeautifulSoup(page_content, "html.parser")

        # 抓取日期元素
        date_elements = soup.select("div._67tN.theme-meta time")

        # 抓取標題元素
        title_elements = soup.select("div._1xc2 h3")

        # 將日期和標題對應成字典，然後添加到列表中
        for date_element, title_element in zip(date_elements, title_elements):
            data23_1 = {
                'date': date_element.text,
                'title': title_element.text
            }
            data_list23_1.append(data23_1)

        i=i+1
        if temp_div_3 == 7:
            temp_div_3 = 1
            if temp_div_2 == 7:
                temp_div_2 = 2
                if temp_div_1 == 2:
                    nextpage = driver.find_element(By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]')
                    nextpage.click()
                    temp_div_1 = 1
                else:
                    temp_div_1 = temp_div_1 + 1
            else:
                temp_div_2 = temp_div_2 + 1
        else:
            temp_div_3 = temp_div_3 + 1

        # 此輪迴圈結束後，將暫時的變數賦值給 div_1、div_2 和 div_3
        div_1 = temp_div_1
        div_2 = temp_div_2
        div_3 = temp_div_3
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
    except NoSuchElementException:
        if(div_3==7):
            div_3=1
            if(div_2==7):
                div_2=2
                if(div_1==2):
                    nextpage=driver.find_element(By.XPATH,'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]').click()
                    div_1=1
                else:
                    div_1=div_1+1
            else:
                div_2=div_2+1
        else:
            div_3=div_3+1
        date3=driver.find_element(By.CLASS_NAME,'_Qfx4')
        date3.click()
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
driver.close()
# 輸出所有日期和標題對應的資料
for data23_1 in data_list23_1:
    print(data23_1['date'], data23_1['title'])
            
with open('新聞標題(2022~2023)/news_23_1.csv', mode='w', encoding='utf-8', newline='') as csvfile:
    # 定義欄位
    fieldnames = ['date', 'title']

    # 將 dictionary 寫入 CSV 檔
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # 寫入第一列的欄位名稱
    writer.writeheader()
    for data23_1 in data_list23_1:
        # 寫入資料
        writer.writerow({'date': data23_1['date'], 'title': data23_1['title']})

221
222
223
224
225
226
227
231
232
233
234
235
236
237
241
242
243
244
245
246
247
251
252
253
254
255
256
257
261
262
263
01/01 CES展登場、台股開紅盤、2022年出口統計 本周大事預告
01/01 〈觀察〉服務業大缺工 內需復甦力道恐遭壓抑
01/01 封關前空方勢化解條件，景氣燈號"藍"燈帶來的機會
01/01 台股逆風而行，2023「紅包行情」鎖定三大選股法則
01/02 〈鴻海子公司併購〉併PRETTL SWH年營收估貢獻4億歐元 毛利率同攀升
01/02 〈鴻海子公司併購〉鴻騰斥61億元併德商 擴充車用線束布局
01/02 帆宣在手訂單維持600億元 今年營收再拚新高
01/02 〈台股風向球〉元月開紅盤消息面平平 靠個股輪流點火紅包行情
01/02 電信三雄佈千座基站 跨年夜全台數據量翻倍創高
01/02 舒緩鄭州廠人力吃緊 鴻海啟動周邊單位支援計畫
01/02 北美暴雪肆虐 台AM零件廠估受惠一好一壞
01/02 顯示器廠搶軌道交通商機 友達拿下全球逾3成市占
01/02 〈觀察〉感測器成長趨勢明確 被動元件廠擴大布局
01/02 12月出口連4黑成定局 Q4出口旺季不旺
01/02 市場需求帶動 材料-KY去年Q4單季營收可望連三季登頂
01/03 中國寶鋼2月續開漲盤 中鋼盤價添底氣
01/03 違規處分全家股票遭罰200萬元 泰山回應了
01/03 農曆春節休市長達12日 期交所擬調高4類契約保證金10%
01/03 豐達科去年12月營收創39個月高 全年增5成
01/03 台股2022年IPO家數、金額雙升 資誠看好半導體今年續航力
01/03 強化資本市場競爭力 金管會今年為台股射6箭
01/03 新光三越拚新春買氣 卡利High檔期日均業績目標創新高
01/03 金融業去年現增逾2100億元 史上最大增資潮今年未完待續
01/03 信邦去年營收308.86億元連13年創高 Q4也創單季最佳
01/03 合併頻譜超標議題 台灣大：繳回將成國庫損失 盼一致性審核
01/03 新鉅科營運不敵需求下滑 去年營收衰退34.1%
01/03 大型專案進補 互動12月營收5.78億元月增247%
01/03 金管會送外銀拚淨

In [10]:
#2023 2月
driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
data_list23_2 = []
data23_2={}#使用大刮號{}創建字典
url='https://news.cnyes.com/news/cat/tw_stock_news?exp=a'
driver.get(url)

date=driver.find_element(By.CLASS_NAME,'_Qfx4')
date.click()
div_1=2 #1or2
div_2=2 #2~7
div_3=4 #1~7
i=0;
#日期往前按8頁
for i in range(8):
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[1]/div[1]/div[1]'))).click()
#2月
for i in range(28):
    try:
        # 初始化列表用於存儲日期和標題
        data_list = []
        print(f"{div_1}{div_2}{div_3}")
        if(i!=0):
            # 使用 JavaScript 進行滾動
            driver.execute_script("window.scrollBy(0, 300);")
            # 選擇日期選擇器
            date_selector = driver.find_element(By.CLASS_NAME, '_Qfx4')
            date_selector.click()
        # 暫時的變數
        temp_div_1 = div_1
        temp_div_2 = div_2
        temp_div_3 = div_3

        # 選擇2/1    
        start = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(start))
        start2 = driver.find_element(*start)
        action = ActionChains(driver)
        action.move_to_element(start2).click().perform()
        # 選擇2/1
        end = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(end))
        end2 = driver.find_element(*end)
        action2 = ActionChains(driver)
        action2.move_to_element(end2).click().perform()
        #確定按鈕
        button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[2]/button[1]')
        driver.execute_script("arguments[0].click();", button)

        time.sleep(5)

    
#模擬向下捲動，以確保載入更多內容
        scrolls = 3
        for _ in range(scrolls):
            driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
            time.sleep(2)  # 等待頁面載入
#找到當前頁面
        page_content=driver.page_source

# 使用 BeautifulSoup 解析頁面内容
        soup = BeautifulSoup(page_content, "html.parser")

        # 抓取日期元素
        date_elements = soup.select("div._67tN.theme-meta time")

        # 抓取標題元素
        title_elements = soup.select("div._1xc2 h3")

        # 將日期和標題對應成字典，然後添加到列表中
        for date_element, title_element in zip(date_elements, title_elements):
            data23_2 = {
                'date': date_element.text,
                'title': title_element.text
            }
            data_list23_2.append(data23_2)

        i=i+1
        if temp_div_3 == 7:
            temp_div_3 = 1
            if temp_div_2 == 7:
                temp_div_2 = 2
                if temp_div_1 == 2:
                    nextpage = driver.find_element(By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]')
                    nextpage.click()
                    temp_div_1 = 1
                else:
                    temp_div_1 = temp_div_1 + 1
            else:
                temp_div_2 = temp_div_2 + 1
        else:
            temp_div_3 = temp_div_3 + 1

        # 此輪迴圈結束後，將暫時的變數賦值給 div_1、div_2 和 div_3
        div_1 = temp_div_1
        div_2 = temp_div_2
        div_3 = temp_div_3
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
    except NoSuchElementException:
        if(div_3==7):
            div_3=1
            if(div_2==7):
                div_2=2
                if(div_1==2):
                    nextpage=driver.find_element(By.XPATH,'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]').click()
                    div_1=1
                else:
                    div_1=div_1+1
            else:
                div_2=div_2+1
        else:
            div_3=div_3+1
        date3=driver.find_element(By.CLASS_NAME,'_Qfx4')
        date3.click()
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
driver.close()
# 輸出所有日期和標題對應的資料
for data23_2 in data_list23_2:
    print(data23_2['date'], data23_2['title'])
            
with open('新聞標題(2022~2023)/news_23_2.csv', mode='w', encoding='utf-8', newline='') as csvfile:
    # 定義欄位
    fieldnames = ['date', 'title']

    # 將 dictionary 寫入 CSV 檔
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # 寫入第一列的欄位名稱
    writer.writeheader()
    for data23_2 in data_list23_2:
        # 寫入資料
        writer.writerow({'date': data23_2['date'], 'title': data23_2['title']})

224
225
226
227
231
232
233
234
235
236
237
241
242
243
244
245
246
247
251
252
253
254
255
256
257
261
262
263
02/01 新制勞退、勞保基金除了台積電 也愛這3檔金融股
02/01 彭博性別平等指數 穩懋等16家台股企業上榜稱霸亞太市場
02/01 富邦金控榮登《Brand Finance》全球500大最有價值品牌及永續指數
02/01 磊山保經今年拚業績目標成長30% 新增1500億元壽險保額建構家庭保障
02/01 愛德萬併購台PCB供應商 拓展測試打樣業務
02/01 新財長就定位 新會期優先推動三大法案「要讓年輕人有感」
02/01 致茂告惠特侵權敗訴 將再提上訴
02/01 仁新晚期乾性黃斑部病變新藥 向美申請三期臨床試驗
02/01 台北國際電玩展來臨 智冠率網龍等團隊聯合參展
02/01 〈長佳智能轉上櫃〉3月上旬轉掛牌 瞄準醫療保健AI市場
02/01 康寧去年每股賺2.09美元 Q1玻璃價格看持平
02/01 〈房產〉觀望氣氛籠罩 12月房貸、土建融年增率雙創波段低
02/01 〈聯亞法說〉Q1營收估持平去年同期 美矽光客戶擴大800G拉貨
02/01 外資連三日加碼聯電 累計張數衝破12.3萬張
02/01 〈長佳智能轉上櫃〉明年首季前拿下多張認證 力拚損平
02/01 〈勞動基金績效〉股債回穩 1月勞動基金收益數可望回升至1500億元
02/01 〈車市〉買氣M型化 每賣出10台車有1台是雙B
02/01 大漲後的激情 2月股市關鍵時間點出爐!
02/01 迅得1月營收5.32億元創同期新高 今年前三季產能滿載
02/01 兔年開春多舛 淘帝-KY董座涉嫌掏空遭起訴
02/01 台股劇情火熱，持續看好當家花旦《中小型股》
02/01 選股不選市 選股雙主軸，雙周報多檔發威
02/01 〈聯亞法說〉Q4純益季增14% 全年每股賺3.61元
02/01 CUBE卡流通卡數突破280萬張創紀錄 國泰世華蟬聯2022年刷卡王功臣
02/01 【量大強漲股整理】~外資不怕今晚聯準會升1碼，連12買超2162億元!
02/01 海能風電47支風機全數安裝完成 逾8成風機開始送電
02/01 富邦Costco聯名卡四大權益升級 拚今年發200萬卡

In [11]:
#2023 3月
driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
data_list23_3 = []
data23_3={}#使用大刮號{}創建字典
url='https://news.cnyes.com/news/cat/tw_stock_news?exp=a'
driver.get(url)

date=driver.find_element(By.CLASS_NAME,'_Qfx4')
date.click()
div_1=2 #1or2
div_2=2 #2~7
div_3=4 #1~7
i=0;
#日期往前按7頁
for i in range(7):
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[1]/div[1]/div[1]'))).click()
#3月
for i in range(31):
    try:
        # 初始化列表用於存儲日期和標題
        data_list = []
        print(f"{div_1}{div_2}{div_3}")
        if(i!=0):
            # 使用 JavaScript 進行滾動
            driver.execute_script("window.scrollBy(0, 300);")
            # 選擇日期選擇器
            date_selector = driver.find_element(By.CLASS_NAME, '_Qfx4')
            date_selector.click()
        # 暫時的變數
        temp_div_1 = div_1
        temp_div_2 = div_2
        temp_div_3 = div_3

        # 選擇3/1    
        start = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(start))
        start2 = driver.find_element(*start)
        action = ActionChains(driver)
        action.move_to_element(start2).click().perform()
        # 選擇3/1
        end = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(end))
        end2 = driver.find_element(*end)
        action2 = ActionChains(driver)
        action2.move_to_element(end2).click().perform()
        #確定按鈕
        button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[2]/button[1]')
        driver.execute_script("arguments[0].click();", button)

        time.sleep(5)

    
#模擬向下捲動，以確保載入更多內容
        scrolls = 3
        for _ in range(scrolls):
            driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
            time.sleep(2)  # 等待頁面載入
#找到當前頁面
        page_content=driver.page_source

# 使用 BeautifulSoup 解析頁面内容
        soup = BeautifulSoup(page_content, "html.parser")

        # 抓取日期元素
        date_elements = soup.select("div._67tN.theme-meta time")

        # 抓取標題元素
        title_elements = soup.select("div._1xc2 h3")

        # 將日期和標題對應成字典，然後添加到列表中
        for date_element, title_element in zip(date_elements, title_elements):
            data23_3 = {
                'date': date_element.text,
                'title': title_element.text
            }
            data_list23_3.append(data23_3)

        i=i+1
        if temp_div_3 == 7:
            temp_div_3 = 1
            if temp_div_2 == 7:
                temp_div_2 = 2
                if temp_div_1 == 2:
                    nextpage = driver.find_element(By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]')
                    nextpage.click()
                    temp_div_1 = 1
                else:
                    temp_div_1 = temp_div_1 + 1
            else:
                temp_div_2 = temp_div_2 + 1
        else:
            temp_div_3 = temp_div_3 + 1

        # 此輪迴圈結束後，將暫時的變數賦值給 div_1、div_2 和 div_3
        div_1 = temp_div_1
        div_2 = temp_div_2
        div_3 = temp_div_3
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
    except NoSuchElementException:
        if(div_3==7):
            div_3=1
            if(div_2==7):
                div_2=2
                if(div_1==2):
                    nextpage=driver.find_element(By.XPATH,'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]').click()
                    div_1=1
                else:
                    div_1=div_1+1
            else:
                div_2=div_2+1
        else:
            div_3=div_3+1
        date3=driver.find_element(By.CLASS_NAME,'_Qfx4')
        date3.click()
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
driver.close()
# 輸出所有日期和標題對應的資料
for data23_3 in data_list23_3:
    print(data23_3['date'], data23_3['title'])
            
with open('新聞標題(2022~2023)/news_23_3.csv', mode='w', encoding='utf-8', newline='') as csvfile:
    # 定義欄位
    fieldnames = ['date', 'title']

    # 將 dictionary 寫入 CSV 檔
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # 寫入第一列的欄位名稱
    writer.writeheader()
    for data23_3 in data_list23_3:
        # 寫入資料
        writer.writerow({'date': data23_3['date'], 'title': data23_3['title']})

224
225
226
227
231
232
233
234
235
236
237
241
242
243
244
245
246
247
251
252
253
254
255
256
257
261
262
263
264
265
266
03/01 〈研華法說〉擴大北美布局 加州新總部4月動工2025年遷入
03/01 〈研華法說〉研華Q1營收拚雙增 雙率可望維持上季水準
03/01 〈研華法說〉宣示啟動第三階段轉型 2030年營收衝50億美元
03/01 今年民參公建商機估逾1700億元 促參金額有望創進10年新高
03/01 〈MWC通訊大會〉高通攜手小米、OPPO等業者 推出全新XR裝置
03/01 〈MWC通訊大會〉明泰秀5G肌肉 友訊展出物聯網智慧藍圖
03/01 台泥年度徵才起跑 釋出九大領域職缺 同步啟動海外人才派任計畫
03/01 普誠去年Q4獲利衝12年新高 全年EPS 0.93元
03/01 毅嘉2月營收登9年同期新高 前2月11.98億元
03/01 避免銀行淪為信用卡個資外洩幫兇 金管會出手管了
03/01 〈MWC通訊大會〉廣達攻電信商機 英業達、和碩秀智慧工廠應用
03/01 〈MWC通訊大會〉盛達推5G解決方案 光寶科展示三大專網應用
03/01 美時治療局部晚期非小細胞肺癌用藥 進軍越南市場
03/01 康普去年EPS 4.72元創五年高 每股擬配3元現金股利
03/01 〈房產〉1月購屋貸款餘額縮水百億元 6年來首見
03/01 內外資聯手砍華新4.2萬張 外資敲進晶圓雙雄
03/01 〈睿生光電轉上市〉擴大切入工業應用、寵物平台 估下半年開始貢獻
03/01 玻纖廠富喬去年第四季落入虧損 全年每股純益0.07元
03/01 群電去年每股賺8.2元 今年營運可望再創高峰
03/01 康普擬斥9.47億元 前往越南建置電池材料產線
03/01 仲琦非北美市場佈局有成 今年營收將再創新高
03/01 〈睿生光電轉上市〉今年營運拚優產業平均 最快本季轉上市
03/01 新藥開發有進展 全福三期臨床收首位病患 永立榮獲美專利
03/01 〈明泰法說〉搶公網、私網商機 今年5G設備營收將翻倍增
03/01 大阪捷絲旅創晶華新里程碑 潘思亮開啟海外版圖
03/01 台灣首檔 ESG 低碳 ETF 核准成立 募集 48.2 億元創近年新

In [12]:
#2023 4月
driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
data_list23_4 = []
data23_4={}#使用大刮號{}創建字典
url='https://news.cnyes.com/news/cat/tw_stock_news?exp=a'
driver.get(url)

date=driver.find_element(By.CLASS_NAME,'_Qfx4')
date.click()
div_1=2 #1or2
div_2=2 #2~7
div_3=7 #1~7
i=0;
#日期往前按6頁
for i in range(6):
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[1]/div[1]/div[1]'))).click()
#4月
for i in range(30):
    try:
        # 初始化列表用於存儲日期和標題
        data_list = []
        print(f"{div_1}{div_2}{div_3}")
        if(i!=0):
            # 使用 JavaScript 進行滾動
            driver.execute_script("window.scrollBy(0, 300);")
            # 選擇日期選擇器
            date_selector = driver.find_element(By.CLASS_NAME, '_Qfx4')
            date_selector.click()
        # 暫時的變數
        temp_div_1 = div_1
        temp_div_2 = div_2
        temp_div_3 = div_3

        # 選擇4/1    
        start = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(start))
        start2 = driver.find_element(*start)
        action = ActionChains(driver)
        action.move_to_element(start2).click().perform()
        # 選擇4/1
        end = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(end))
        end2 = driver.find_element(*end)
        action2 = ActionChains(driver)
        action2.move_to_element(end2).click().perform()
        #確定按鈕
        button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[2]/button[1]')
        driver.execute_script("arguments[0].click();", button)

        time.sleep(5)

    
#模擬向下捲動，以確保載入更多內容
        scrolls = 3
        for _ in range(scrolls):
            driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
            time.sleep(2)  # 等待頁面載入
#找到當前頁面
        page_content=driver.page_source

# 使用 BeautifulSoup 解析頁面内容
        soup = BeautifulSoup(page_content, "html.parser")

        # 抓取日期元素
        date_elements = soup.select("div._67tN.theme-meta time")

        # 抓取標題元素
        title_elements = soup.select("div._1xc2 h3")

        # 將日期和標題對應成字典，然後添加到列表中
        for date_element, title_element in zip(date_elements, title_elements):
            data23_4 = {
                'date': date_element.text,
                'title': title_element.text
            }
            data_list23_4.append(data23_4)

        i=i+1
        if temp_div_3 == 7:
            temp_div_3 = 1
            if temp_div_2 == 7:
                temp_div_2 = 2
                if temp_div_1 == 2:
                    nextpage = driver.find_element(By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]')
                    nextpage.click()
                    temp_div_1 = 1
                else:
                    temp_div_1 = temp_div_1 + 1
            else:
                temp_div_2 = temp_div_2 + 1
        else:
            temp_div_3 = temp_div_3 + 1

        # 此輪迴圈結束後，將暫時的變數賦值給 div_1、div_2 和 div_3
        div_1 = temp_div_1
        div_2 = temp_div_2
        div_3 = temp_div_3
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
    except NoSuchElementException:
        if(div_3==7):
            div_3=1
            if(div_2==7):
                div_2=2
                if(div_1==2):
                    nextpage=driver.find_element(By.XPATH,'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]').click()
                    div_1=1
                else:
                    div_1=div_1+1
            else:
                div_2=div_2+1
        else:
            div_3=div_3+1
        date3=driver.find_element(By.CLASS_NAME,'_Qfx4')
        date3.click()
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
driver.close()
# 輸出所有日期和標題對應的資料
for data23_4 in data_list23_4:
    print(data23_4['date'], data23_4['title'])
            
with open('新聞標題(2022~2023)/news_23_4.csv', mode='w', encoding='utf-8', newline='') as csvfile:
    # 定義欄位
    fieldnames = ['date', 'title']

    # 將 dictionary 寫入 CSV 檔
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # 寫入第一列的欄位名稱
    writer.writeheader()
    for data23_4 in data_list23_4:
        # 寫入資料
        writer.writerow({'date': data23_4['date'], 'title': data23_4['title']})

227
231
232
233
234
235
236
237
241
242
243
244
245
246
247
251
252
253
254
255
256
257
261
262
263
264
265
266
267
271
04/01 劉德音開講、IC設計白皮書、永豐金法說 本周大事回顧
04/01 〈潛力股〉新事業逐步貢獻 正道迎新一波成長
04/01 〈潛力股〉iPhone 15旗艦機種改採Type-C 台鏈Q2起迎備貨潮
04/01 〈潛力股〉佐登-KY今年重回成長 獲利拚賺逾半個股本
04/01 〈潛力股〉彰銀預告今年股利不縮水 有望吸引買盤敲進
04/01 〈潛力股〉茂順滿手高毛利訂單 營收及獲利將逐季成長
04/01 〈潛力股〉漢來美食喜迎最大展店潮 餐飲落後補漲效應可期
04/02 〈房產〉南台灣唯一 日建築師隈研吾旋風來台公開在南台灣規劃建案
04/02 〈觀察〉鋼鐵供需持續改善 鐵礦砂短期震盪無礙旺季行情
04/02 〈觀察〉銀行重啟海外版圖拓點 目標鎖定三大市場
04/02 〈觀察〉沖壓件廠新產能今年陸續到位 鎖定半導體及車用需求
04/02 〈觀察〉餐飲集團競相插旗美國為哪樁？
04/03 鴻海、大立光Q1營收出爐 多家台廠法說 本周大事預告
04/03 〈Q2產業前瞻〉全球加速淨零減排 機電廠高效節能馬達迎汰換潮
04/03 〈Q2產業前瞻〉基建需求、5G設備升級撐腰 網通市況先蹲後跳
04/03 景氣放緩干擾大盤 台股Q2回穩要看庫存與獲利臉色
04/03 〈獨家〉健鼎以1美元完成購併日商在越PCB廠 股權完成移轉
04/03 終端去庫存時間恐延長？智庫看法不同調
04/03 〈研之有物〉語言只是工具？別忘了感知複雜的自己！鄭毓瑜院士談漢字轉型
04/04 長榮股東會市價600元康寧隨行杯大方送 近23萬名零股股東想領有兩招
04/04 〈Q2產業前瞻〉紡織景氣脫離谷底 成衣、化纖廠營運回溫
04/04 〈Q2產業前瞻〉新藥廠陸續完成收案 推進臨床試驗
04/04 逸達過敏性氣喘新藥二期臨床 主要指標數據正向
04/04 〈Q2產業前瞻〉矽光啟動新傳輸革命 800G交換器將進入CPO時代
04/04 〈Q2產業前瞻〉PCB廠Q2能見度低 靠汽車板及低軌衛星板訂單支撐
04/04 〈Q2產業前瞻〉出入境旅遊熱度升 旅遊業榮景來

In [14]:
#2023 5月
driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
data_list23_5 = []
data23_5={}#使用大刮號{}創建字典
url='https://news.cnyes.com/news/cat/tw_stock_news?exp=a'
driver.get(url)

date=driver.find_element(By.CLASS_NAME,'_Qfx4')
date.click()
div_1=2 #1or2
div_2=2 #2~7
div_3=2 #1~7
i=0;
#日期往前按5頁
for i in range(5):
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[1]/div[1]/div[1]'))).click()
#5月
for i in range(31):
    try:
        # 初始化列表用於存儲日期和標題
        data_list = []
        print(f"{div_1}{div_2}{div_3}")
        if(i!=0):
            # 使用 JavaScript 進行滾動
            driver.execute_script("window.scrollBy(0, 300);")
            # 選擇日期選擇器
            date_selector = driver.find_element(By.CLASS_NAME, '_Qfx4')
            date_selector.click()
        # 暫時的變數
        temp_div_1 = div_1
        temp_div_2 = div_2
        temp_div_3 = div_3

        # 選擇5/1    
        start = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(start))
        start2 = driver.find_element(*start)
        action = ActionChains(driver)
        action.move_to_element(start2).click().perform()
        # 選擇5/1
        end = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(end))
        end2 = driver.find_element(*end)
        action2 = ActionChains(driver)
        action2.move_to_element(end2).click().perform()
        #確定按鈕
        button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[2]/button[1]')
        driver.execute_script("arguments[0].click();", button)

        time.sleep(5)

    
#模擬向下捲動，以確保載入更多內容
        scrolls = 3
        for _ in range(scrolls):
            driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
            time.sleep(2)  # 等待頁面載入
#找到當前頁面
        page_content=driver.page_source

# 使用 BeautifulSoup 解析頁面内容
        soup = BeautifulSoup(page_content, "html.parser")

        # 抓取日期元素
        date_elements = soup.select("div._67tN.theme-meta time")

        # 抓取標題元素
        title_elements = soup.select("div._1xc2 h3")

        # 將日期和標題對應成字典，然後添加到列表中
        for date_element, title_element in zip(date_elements, title_elements):
            data23_5 = {
                'date': date_element.text,
                'title': title_element.text
            }
            data_list23_5.append(data23_5)

        i=i+1
        if temp_div_3 == 7:
            temp_div_3 = 1
            if temp_div_2 == 7:
                temp_div_2 = 2
                if temp_div_1 == 2:
                    nextpage = driver.find_element(By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]')
                    nextpage.click()
                    temp_div_1 = 1
                else:
                    temp_div_1 = temp_div_1 + 1
            else:
                temp_div_2 = temp_div_2 + 1
        else:
            temp_div_3 = temp_div_3 + 1

        # 此輪迴圈結束後，將暫時的變數賦值給 div_1、div_2 和 div_3
        div_1 = temp_div_1
        div_2 = temp_div_2
        div_3 = temp_div_3
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
    except NoSuchElementException:
        if(div_3==7):
            div_3=1
            if(div_2==7):
                div_2=2
                if(div_1==2):
                    nextpage=driver.find_element(By.XPATH,'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]').click()
                    div_1=1
                else:
                    div_1=div_1+1
            else:
                div_2=div_2+1
        else:
            div_3=div_3+1
        date3=driver.find_element(By.CLASS_NAME,'_Qfx4')
        date3.click()
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
driver.close()
# 輸出所有日期和標題對應的資料
for data23_5 in data_list23_5:
    print(data23_5['date'], data23_5['title'])
            
with open('新聞標題(2022~2023)/news_23_5.csv', mode='w', encoding='utf-8', newline='') as csvfile:
    # 定義欄位
    fieldnames = ['date', 'title']

    # 將 dictionary 寫入 CSV 檔
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # 寫入第一列的欄位名稱
    writer.writeheader()
    for data23_5 in data_list23_5:
        # 寫入資料
        writer.writerow({'date': data23_5['date'], 'title': data23_5['title']})

222
223
224
225
226
227
231
232
233
234
235
236
237
241
242
243
244
245
246
247
251
252
253
254
255
256
257
261
262
263
264
05/01 股市回升加持 勞動基金3月收益數2190億元
05/01 永信建設首季財報出爐EPS 1.19元 今年將有6個建案完工銷售
05/01 亞銀理事年會5/2韓國仁川登場 財長率團出訪聚焦2議題
05/01 車市好熱鬧！這些日、德系新車齊發 有望一路旺至年底
05/01 駁郭台銘的「生肺癌說」經長：2016年後空污降比逾6成
05/01 快找發票對獎 去年11-12月千萬發票還有3張沒領
05/01 〈台股風向球〉本周美股影響力增 別小看傳統「五窮」魔咒
05/01 〈華邦電展望〉20奈米DRAM良率進展優預期 可望提前半年量產
05/01 〈華邦電展望〉系統廠庫存見底 急單湧入需求逐月增 本季營收增幅上看15%
05/01 智擎胰臟癌新藥一線用藥有望明年上市 全球銷量將成長2倍
05/01 台版晶片法子法下午預告 財長：研發費用門檻60億元
05/01 需求降溫 景碩Q1毛利率探兩年低點 EPS 0.02元
05/02 國產汐止二廠動土 明年完工年產能增30萬立方米
05/02 廢鋼跌勢擴大 豐興鋼筋每公噸跌千元
05/02 台股4月震盪 拖累權證發行檔數、金額雙降
05/02 華碩血壓應用軟體通過衛福部認證 將導入VivoWatch智慧手錶
05/02 借貸平台imB涉非法吸金25億元 金管會示警四大風險
05/02 以創意、大世科驗證投資池的重要性！3.0即將改版！
05/02 台股年輕化！4月新增開戶數逾2.7萬戶 「20代」月增占超過一半
05/02 中碳Q1每股賺1.4元 下半年可望優於上半年
05/02 雲豹能源200MW儲能案場動工 今年營收戰百億元
05/02 嘉聯益首季稼動率偏低 單季大虧8.56億元 每股虧損1.55元
05/02 崇越電Q1 EPS 0.68元 估Q2營運仍在谷底
05/02 智邦宣示2030年碳排減半 2050年達淨零碳排
05/02 〈環球晶法說〉徐秀蘭：Q2營運面臨挑戰 下半年需求可望復甦
05/02 致伸Q1毛利率、純益雙創同期高 EPS 1.23元
05/02 群電

In [15]:
#2023 6月
driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
data_list23_6 = []
data23_6={}#使用大刮號{}創建字典
url='https://news.cnyes.com/news/cat/tw_stock_news?exp=a'
driver.get(url)

date=driver.find_element(By.CLASS_NAME,'_Qfx4')
date.click()
div_1=2 #1or2
div_2=2 #2~7
div_3=5 #1~7
i=0;
#日期往前按4頁
for i in range(4):
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[1]/div[1]/div[1]'))).click()
#6月
for i in range(30):
    try:
        # 初始化列表用於存儲日期和標題
        data_list = []
        print(f"{div_1}{div_2}{div_3}")
        if(i!=0):
            # 使用 JavaScript 進行滾動
            driver.execute_script("window.scrollBy(0, 300);")
            # 選擇日期選擇器
            date_selector = driver.find_element(By.CLASS_NAME, '_Qfx4')
            date_selector.click()
        # 暫時的變數
        temp_div_1 = div_1
        temp_div_2 = div_2
        temp_div_3 = div_3

        # 選擇6/1    
        start = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(start))
        start2 = driver.find_element(*start)
        action = ActionChains(driver)
        action.move_to_element(start2).click().perform()
        # 選擇6/1
        end = (By.XPATH, f'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[{temp_div_1}]/div[{temp_div_2}]/div[{temp_div_3}]/div')
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(end))
        end2 = driver.find_element(*end)
        action2 = ActionChains(driver)
        action2.move_to_element(end2).click().perform()
        #確定按鈕
        button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[2]/button[1]')
        driver.execute_script("arguments[0].click();", button)

        time.sleep(5)

    
#模擬向下捲動，以確保載入更多內容
        scrolls = 3
        for _ in range(scrolls):
            driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
            time.sleep(2)  # 等待頁面載入
#找到當前頁面
        page_content=driver.page_source

# 使用 BeautifulSoup 解析頁面内容
        soup = BeautifulSoup(page_content, "html.parser")

        # 抓取日期元素
        date_elements = soup.select("div._67tN.theme-meta time")

        # 抓取標題元素
        title_elements = soup.select("div._1xc2 h3")

        # 將日期和標題對應成字典，然後添加到列表中
        for date_element, title_element in zip(date_elements, title_elements):
            data23_6 = {
                'date': date_element.text,
                'title': title_element.text
            }
            data_list23_6.append(data23_6)

        i=i+1
        if temp_div_3 == 7:
            temp_div_3 = 1
            if temp_div_2 == 7:
                temp_div_2 = 2
                if temp_div_1 == 2:
                    nextpage = driver.find_element(By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]')
                    nextpage.click()
                    temp_div_1 = 1
                else:
                    temp_div_1 = temp_div_1 + 1
            else:
                temp_div_2 = temp_div_2 + 1
        else:
            temp_div_3 = temp_div_3 + 1

        # 此輪迴圈結束後，將暫時的變數賦值給 div_1、div_2 和 div_3
        div_1 = temp_div_1
        div_2 = temp_div_2
        div_3 = temp_div_3
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
    except NoSuchElementException:
        if(div_3==7):
            div_3=1
            if(div_2==7):
                div_2=2
                if(div_1==2):
                    nextpage=driver.find_element(By.XPATH,'//*[@id="content"]/div/div/div[2]/main/div[3]/div[1]/nav/div/div/div[1]/div[2]/div[1]/div[3]').click()
                    div_1=1
                else:
                    div_1=div_1+1
            else:
                div_2=div_2+1
        else:
            div_3=div_3+1
        date3=driver.find_element(By.CLASS_NAME,'_Qfx4')
        date3.click()
        # 滾動網頁到最上層
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
driver.close()
# 輸出所有日期和標題對應的資料
for data23_6 in data_list23_6:
    print(data23_6['date'], data23_6['title'])
            
with open('新聞標題(2022~2023)/news_23_6.csv', mode='w', encoding='utf-8', newline='') as csvfile:
    # 定義欄位
    fieldnames = ['date', 'title']

    # 將 dictionary 寫入 CSV 檔
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # 寫入第一列的欄位名稱
    writer.writeheader()
    for data23_6 in data_list23_6:
        # 寫入資料
        writer.writerow({'date': data23_6['date'], 'title': data23_6['title']})

225
226
227
231
232
233
234
235
236
237
241
242
243
244
245
246
247
251
252
253
254
255
256
257
261
262
263
264
265
266
06/01 台美簽署貿易倡議首批協議 「台美中小企業諮詢會」3面向助攻
06/01 中鋼新進人員甄試報名開跑 今年預計招募243人
06/01 十銓前4月可望轉盈 明年工控營收占比衝1成
06/01 薪資、股利雙升 2022年綜所稅收淨額2440億元、年增4成
06/01 鴻海5G產品布局新斬獲 取得O-RAN聯盟OTIC室外RU認證
06/01 南科發生壓降 台積電預期不影響營運、聯電部分晶圓報廢
06/01 印度半導體投資傳卡關 鴻海：持續與政府溝通
06/01 壽險業入不敷出？公會澄清：現金水位充裕 保單解約已大幅改善
06/01 毅嘉車用產品出貨旺 累計營收33億元創9年新高
06/01 惠珊投資池：僑威、中磊、華星光、星通、先進光
06/01 上市公司員工酬勞前10名 半導體包辦8名次 日月光550.8萬元稱雄
06/01 澳豐基金倒閉、萬人受害 金管會：13家上市櫃公司也踩雷
06/01 台嘉碩估Q2景氣溫和復甦 每股2.5元股息訂7/17除息
06/01 全球25大最佳航空公司 長榮航拿下台灣唯一名次
06/01 設備廠迅得接單強勁 估第二季營收將挑戰歷史新高
06/01 〈台幣〉股匯脫鉤 新台幣小升2.6分收30.740元
06/01 5月我製造業PMI連3跌 六大產業看未來半年全數緊縮
06/01 外資搶「機票」！加碼航空雙雄 連5日買超長榮航近12萬張
06/01 長聖5月營收續創同期高 細胞委託訂單直達12月
06/01 瑩碩精神科用藥中國藥證到手 搶攻人民幣1億元商機
06/01 日月光搶AI商機 推FOCoS-Bridge整合多顆ASIC
06/01 量價齊跌 Q1 Enterprise SSD營收季減近5成
06/01 LEXUS踩油門衝單月銷售亞軍 5月銷量飆增逾130%
06/01 LED照明需求回升 帶動全年LED晶片產值回升5%
06/01 中美晶攜手日立冷氣 打造台灣空調業界最大單一太陽能場域
06/01 永豐金將AI列最高決策 全體員工禁止做兩件事嚴控風險
06/01 陳俊聖：PC產業庫存鬧